# Academia

## 1. MISQ.csv 만들기
- 2023년 ~ 2025년, 3월, 6월, 9월, 12월로 나눠서 date 저장
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 MISQ로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [1]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정: 여기만 수정하세요
# ========================================
VOL_NUMBERS = [47, 48, 49]  # 처리할 vol 번호 리스트
START_YEAR = 2023  # 첫 번째 vol의 시작 연도
JOURNAL_PREFIX = "misq"  # 파일명 접두사 (예: misq, jmis, isj 등)
# ========================================

# 파일들이 있는 디렉토리 경로
data_dir = "/home/dslab/choi/Journal/Data/Academia/MISQ"

# CSV 파일들 찾기
file_pattern = f"{JOURNAL_PREFIX}_vol*.csv"
csv_files = glob(os.path.join(data_dir, file_pattern))

if not csv_files:
    print(f"⚠ '{file_pattern}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {data_dir}")
    exit()

print(f"✓ {len(csv_files)}개의 {JOURNAL_PREFIX.upper()} 파일 발견\n")

# 날짜 매핑 함수
def get_date(vol, iss):
    """
    vol과 iss 번호로 날짜 생성
    vol: volume 번호
    iss: issue 번호 (1~4)
    """
    # vol 번호에서 연도 계산
    if vol in VOL_NUMBERS:
        year = START_YEAR + (VOL_NUMBERS.index(vol))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (vol - VOL_NUMBERS[0])

    # iss에 따른 월 매핑
    month_mapping = {
        1: "03",
        2: "06",
        3: "09",
        4: "12"
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

for file_path in sorted(csv_files):
    # 파일명에서 vol과 iss 추출
    filename = os.path.basename(file_path)
    match = re.search(r'vol(\d+)_iss(\d+)', filename)

    if match:
        vol_num = int(match.group(1))
        iss_num = int(match.group(2))

        # CSV 파일 읽기
        try:
            df = pd.read_csv(file_path)

            # date 칼럼 추가
            df['date'] = get_date(vol_num, iss_num)

            # affiliations 칼럼 추가 (저널명을 대문자로)
            df['affiliations'] = JOURNAL_PREFIX.upper()

            all_dfs.append(df)

            print(f"처리: {filename:30s} -> date: {df['date'].iloc[0]}, rows: {len(df)}")

        except Exception as e:
            print(f"❌ 오류 ({filename}): {e}")

print(f"\n{'='*60}")

# 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행")

    # 1. null 값 제거 (title, abstract 기준)
    print(f"\n[1단계] Null 값 제거 (title, abstract 기준)")
    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 2. 중복 제거 (title, abstract 기준)
    print(f"\n[2단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 3. 최종 컬럼 선택
    print(f"\n[3단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 4. 날짜별 통계
    print(f"\n[4단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 5. 결과 저장
    output_filename = f"{JOURNAL_PREFIX.upper()}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_PREFIX.upper()}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    print(final_df.head(3).to_string())

else:
    print("❌ 처리할 데이터가 없습니다.")

✓ 11개의 MISQ 파일 발견

처리: misq_vol47_iss1.csv            -> date: 2023-03, rows: 18
처리: misq_vol47_iss2.csv            -> date: 2023-06, rows: 17
처리: misq_vol47_iss3.csv            -> date: 2023-09, rows: 17
처리: misq_vol47_iss4.csv            -> date: 2023-12, rows: 19
처리: misq_vol48_iss1.csv            -> date: 2024-03, rows: 17
처리: misq_vol48_iss2.csv            -> date: 2024-06, rows: 19
처리: misq_vol48_iss3.csv            -> date: 2024-09, rows: 19
처리: misq_vol48_iss4.csv            -> date: 2024-12, rows: 26
처리: misq_vol49_iss1.csv            -> date: 2025-03, rows: 20
처리: misq_vol49_iss2.csv            -> date: 2025-06, rows: 19
처리: misq_vol49_iss3.csv            -> date: 2025-09, rows: 19

✓ 병합 완료: 총 210개 행

[1단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 210개 행
  - 제거 후: 195개 행
  - 제거됨: 15개 행

[2단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 195개 행
  - 제거 후: 194개 행
  - 제거됨: 1개 행

[3단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[4단계] 날짜별 통계
  - 2023-03:

## 2. DSS.csv 만들기
- DSS 폴더에 있는 거 다 병합
- 중복 제거 -> title, abstract 기준
- date는 기존에 있는 거 그대로 사용
- date -> 2023 ~ 2025 아닌 거 삭제
- null 삭제 -> title, abstract 기준
- affiliations 생성 -> DSS로 설정
- 최종 칼럼 -> date, title, abstract, keywords, affiliations

In [2]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "DSS"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/DSS"  # DSS 파일들이 있는 디렉토리
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

# DSS CSV 파일들 찾기 (dss로 시작하는 모든 csv 파일)
file_pattern = "dss*.csv"
csv_files = glob(os.path.join(DATA_DIR, file_pattern))

if not csv_files:
    print(f"⚠ '{file_pattern}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 DSS 파일 발견\n")

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    try:
        df = pd.read_csv(file_path)
        all_dfs.append(df)
        print(f"읽기: {filename:40s} -> {len(df)}개 행")

    except Exception as e:
        print(f"❌ 오류 ({filename}): {e}")

print(f"\n{'='*60}")

# 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행")

    # 1. affiliations 칼럼 추가
    print(f"\n[1단계] affiliations 생성")
    combined_df['affiliations'] = JOURNAL_NAME
    print(f"  - affiliations: {JOURNAL_NAME}")

    # 2. null 값 제거 (title, abstract 기준)
    print(f"\n[2단계] Null 값 제거 (title, abstract 기준)")
    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 3. date 컬럼 확인 및 연도 필터링
    print(f"\n[3단계] Date 필터링 (2023-2025)")

    if 'date' not in combined_df.columns:
        print("  ❌ 'date' 컬럼이 없습니다!")
        exit()

    # date에서 연도 추출
    before_date = len(combined_df)

    # date 형식 확인 (여러 형식 지원)
    def extract_year(date_str):
        """date 문자열에서 연도 추출"""
        if pd.isna(date_str):
            return None

        date_str = str(date_str)
        # YYYY-MM-DD, YYYY-MM, YYYY 등 다양한 형식 지원
        match = re.search(r'(\d{4})', date_str)
        if match:
            return int(match.group(1))
        return None

    combined_df['year'] = combined_df['date'].apply(extract_year)

    # 유효한 연도만 필터링
    combined_df = combined_df[combined_df['year'].isin(VALID_YEARS)]
    combined_df = combined_df.drop('year', axis=1)  # 임시 컬럼 제거

    after_date = len(combined_df)
    print(f"  - 필터링 전: {before_date}개 행")
    print(f"  - 필터링 후: {after_date}개 행")
    print(f"  - 제거됨: {before_date - after_date}개 행")

    # 4. 중복 제거 (title, abstract 기준)
    print(f"\n[4단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 5. 최종 컬럼 선택
    print(f"\n[5단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 6. 날짜별 통계
    print(f"\n[6단계] 날짜별 통계")

    # date를 연도-월 형식으로 변환하여 통계
    def format_date_for_stats(date_str):
        if pd.isna(date_str):
            return 'Unknown'
        date_str = str(date_str)
        # YYYY-MM 형식으로 변환
        match = re.search(r'(\d{4})-?(\d{2})?', date_str)
        if match:
            year = match.group(1)
            month = match.group(2) if match.group(2) else '01'
            return f"{year}-{month}"
        return date_str

    final_df['date_formatted'] = final_df['date'].apply(format_date_for_stats)
    date_stats = final_df.groupby('date_formatted').size().sort_index()
    final_df = final_df.drop('date_formatted', axis=1)

    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 7. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

else:
    print("❌ 처리할 데이터가 없습니다.")

✓ 16개의 DSS 파일 발견

읽기: dss_vol_164_to_166.csv                   -> 16개 행
읽기: dss_vol_165_to_166.csv                   -> 16개 행
읽기: dss_vol_166_to_167.csv                   -> 16개 행
읽기: dss_vol_167_to_167.csv                   -> 8개 행
읽기: dss_vol_168_to_168.csv                   -> 9개 행
읽기: dss_vol_169_to_169.csv                   -> 9개 행
읽기: dss_vol_170_to_170.csv                   -> 10개 행
읽기: dss_vol_171_to_171.csv                   -> 11개 행
읽기: dss_vol_172_to_172.csv                   -> 10개 행
읽기: dss_vol_173_to_173.csv                   -> 10개 행
읽기: dss_vol_174_to_174.csv                   -> 13개 행
읽기: dss_vol_175_to_175.csv                   -> 14개 행
읽기: dss_vol_176_to_176.csv                   -> 14개 행
읽기: dss_vol_176_to_180.csv                   -> 91개 행
읽기: dss_vol_181_to_199.csv                   -> 40개 행
읽기: dss_vol_199_to_199.csv                   -> 8개 행

✓ 병합 완료: 총 295개 행

[1단계] affiliations 생성
  - affiliations: DSS

[2단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 295개 행
  -

## 3.ISR.csv 만들기
- /home/dslab/choi/WITS/원본데이터/ISR.csv 여기에 있는 데이터랑 수집한 데이터 /home/dslab/choi/Journal/Data/Academia/isre_vol36_1.csv이랑 합치기.
- 데이터에 있는 vol 34 ~ 36, iss 1~4을 2023년 ~ 2025년, 3월, 6월, 9월, 12월로 나눠서 date 저장
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 MISQ로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [1]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "ISR"
# 기존 데이터 경로
ORIGINAL_DATA_PATH = "/home/dslab/choi/WITS/원본데이터/ISR.csv"
# 새로 수집한 데이터 경로
NEW_DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/"
NEW_DATA_PATTERN = "isre_vol*.csv"
# vol-year 매핑
VOL_NUMBERS = [34, 35, 36]  # vol 34, 35, 36
START_YEAR = 2023  # vol 34 = 2023년
# ========================================

print(f"{'='*60}")
print(f"ISR.csv 생성 시작")
print(f"{'='*60}\n")

# 데이터프레임을 저장할 리스트
all_dfs = []

# 1. 기존 데이터 읽기
print("[1단계] 기존 데이터 읽기")
if os.path.exists(ORIGINAL_DATA_PATH):
    try:
        original_df = pd.read_csv(ORIGINAL_DATA_PATH)
        all_dfs.append(original_df)
        print(f"  ✓ 기존 데이터: {ORIGINAL_DATA_PATH}")
        print(f"    - 행 수: {len(original_df)}")
    except Exception as e:
        print(f"  ❌ 기존 데이터 읽기 오류: {e}")
else:
    print(f"  ⚠ 기존 데이터 파일이 없습니다: {ORIGINAL_DATA_PATH}")

# 2. 새로 수집한 데이터 읽기
print(f"\n[2단계] 새 데이터 읽기")
new_csv_files = glob(os.path.join(NEW_DATA_DIR, NEW_DATA_PATTERN))

if new_csv_files:
    print(f"  ✓ {len(new_csv_files)}개의 새 데이터 파일 발견")

    # 날짜 매핑 함수
    def get_date(vol, iss):
        """vol과 iss 번호로 날짜 생성"""
        if vol in VOL_NUMBERS:
            year = START_YEAR + (VOL_NUMBERS.index(vol))
        else:
            # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
            year = START_YEAR + (vol - VOL_NUMBERS[0])

        # iss에 따른 월 매핑
        month_mapping = {
            1: "03",
            2: "06",
            3: "09",
            4: "12"
        }

        month = month_mapping.get(iss, "03")
        return f"{year}-{month}"

    for file_path in sorted(new_csv_files):
        filename = os.path.basename(file_path)

        # 파일명에서 vol과 iss 추출
        match = re.search(r'vol(\d+)_(\d+)', filename)

        if match:
            vol_num = int(match.group(1))
            iss_num = int(match.group(2))

            try:
                df = pd.read_csv(file_path)

                # date 칼럼 추가
                df['date'] = get_date(vol_num, iss_num)

                all_dfs.append(df)
                print(f"    - {filename:35s} -> date: {df['date'].iloc[0]}, rows: {len(df)}")

            except Exception as e:
                print(f"    ❌ 오류 ({filename}): {e}")
        else:
            print(f"    ⚠ 파일명 패턴 불일치: {filename}")
else:
    print(f"  ⚠ 새 데이터 파일이 없습니다: {NEW_DATA_DIR}{NEW_DATA_PATTERN}")

print(f"\n{'='*60}")

# 3. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 4. affiliations 칼럼 추가
    print(f"\n[3단계] affiliations 생성")
    combined_df['affiliations'] = JOURNAL_NAME
    print(f"  - affiliations: {JOURNAL_NAME}")

    # 5. null 값 제거 (title, abstract 기준)
    print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")
    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 6. 중복 제거 (title, abstract 기준)
    print(f"\n[5단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 7. 최종 컬럼 선택
    print(f"\n[6단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 8. 날짜별 통계
    print(f"\n[7단계] 날짜별 통계")

    # date가 있는 경우만 통계
    if final_df['date'].notna().any():
        date_stats = final_df.groupby('date').size().sort_index()
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
    else:
        print("  - date 정보가 없습니다.")

    # 9. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ Vol 범위: {min(VOL_NUMBERS)} - {max(VOL_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

else:
    print("❌ 처리할 데이터가 없습니다.")
    print("  - 기존 데이터 경로를 확인하세요.")
    print("  - 새 데이터 경로를 확인하세요.")

ISR.csv 생성 시작

[1단계] 기존 데이터 읽기
  ❌ 기존 데이터 읽기 오류: 'utf-8' codec can't decode byte 0xa1 in position 4683: invalid start byte

[2단계] 새 데이터 읽기
  ✓ 2개의 새 데이터 파일 발견
    - isre_vol36_1.csv                    -> date: 2025-03, rows: 62
    - isre_vol36_3.csv                    -> date: 2025-09, rows: 30

✓ 병합 완료: 총 92개 행 (2개 파일)

[3단계] affiliations 생성
  - affiliations: ISR

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 92개 행
  - 제거 후: 87개 행
  - 제거됨: 5개 행

[5단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 87개 행
  - 제거 후: 58개 행
  - 제거됨: 29개 행

[6단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[7단계] 날짜별 통계
  - 2025-03: 58개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Academia/ISR.csv
✓ 최종 행 수: 58개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 저널: ISR
✓ Vol 범위: 34 - 36
✓ 연도 범위: 2023 - 2025

[데이터 미리보기]
      date                                                                                                                                     

## 4. IAM.csv 만들기
- 폴더에 있는 IAM.csv에서 필요한 컬럼만 저장
- affiliations 컬럼 생성 -> IAM으로 지정
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [2]:
import pandas as pd
import os

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "IAM"
# IAM 데이터 경로 (필요시 수정)
INPUT_FILE_PATH = "/home/dslab/choi/Journal/Data/Academia/IAM/IAM.csv"
# ========================================

print(f"{'='*60}")
print(f"IAM.csv 생성 시작")
print(f"{'='*60}\n")

# 1. IAM 데이터 읽기
print("[1단계] IAM 데이터 읽기")

if not os.path.exists(INPUT_FILE_PATH):
    print(f"  ❌ 파일이 없습니다: {INPUT_FILE_PATH}")
    print(f"  ⚠ 다른 경로에 있다면 코드 상단의 INPUT_FILE_PATH를 수정하세요.")
    exit()

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"  ✓ 파일 읽기 완료: {INPUT_FILE_PATH}")
    print(f"    - 행 수: {len(df)}")
    print(f"    - 컬럼: {list(df.columns)}")
except Exception as e:
    print(f"  ❌ 파일 읽기 오류: {e}")
    exit()

print(f"\n{'='*60}")

# 2. affiliations 컬럼 생성
print(f"\n[2단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 3. null 값 제거 (title, abstract 기준)
print(f"\n[3단계] Null 값 제거 (title, abstract 기준)")

# title과 abstract 컬럼 존재 여부 확인
required_cols = ['title', 'abstract']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 4. 중복 제거 (title, abstract 기준)
print(f"\n[4단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 5. 최종 컬럼 선택
print(f"\n[5단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 6. 날짜별 통계 (date 컬럼이 있는 경우)
if 'date' in df.columns and final_df['date'].notna().any():
    print(f"\n[6단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"\n[6단계] 날짜 정보 없음")

# 7. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 저널: {JOURNAL_NAME}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print("✓ IAM.csv 생성 완료!")
print(f"{'='*60}")

IAM.csv 생성 시작

[1단계] IAM 데이터 읽기
  ✓ 파일 읽기 완료: /home/dslab/choi/Journal/Data/Academia/IAM/IAM.csv
    - 행 수: 298
    - 컬럼: ['volume', 'issue', 'title', 'authors', 'abstract', 'date', 'keywords', 'url']


[2단계] affiliations 생성
  - affiliations: IAM

[3단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 298개 행
  - 제거 후: 270개 행
  - 제거됨: 28개 행

[4단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 270개 행
  - 제거 후: 270개 행
  - 제거됨: 0개 행

[5단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[6단계] 날짜별 통계
  - 총 23개의 날짜
  - 첫 날짜: 2023/01/01 (4개)
  - 마지막 날짜: 2025/12/01 (21개)

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Academia/IAM.csv
✓ 최종 행 수: 270개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 저널: IAM

[데이터 미리보기]
         date                                                                                                                                  title                                                                                                  

## 5. JIT.csv 만들기
- 이전 코드 붙여서 인스턴스만 수정

In [3]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정: 여기만 수정하세요
# ========================================
VOL_NUMBERS = [38, 39, 40]  # 처리할 vol 번호 리스트
START_YEAR = 2023  # 첫 번째 vol의 시작 연도
JOURNAL_PREFIX = "JIT"  # 파일명 접두사 (예: misq, jmis, isj 등)
# ========================================

# 파일들이 있는 디렉토리 경로
data_dir = "/home/dslab/choi/Journal/Data/Academia/JIT"

# CSV 파일들 찾기
file_pattern = f"{JOURNAL_PREFIX}_vol*.csv"
csv_files = glob(os.path.join(data_dir, file_pattern))

if not csv_files:
    print(f"⚠ '{file_pattern}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {data_dir}")
    exit()

print(f"✓ {len(csv_files)}개의 {JOURNAL_PREFIX.upper()} 파일 발견\n")

# 날짜 매핑 함수
def get_date(vol, iss):
    """
    vol과 iss 번호로 날짜 생성
    vol: volume 번호
    iss: issue 번호 (1~4)
    """
    # vol 번호에서 연도 계산
    if vol in VOL_NUMBERS:
        year = START_YEAR + (VOL_NUMBERS.index(vol))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (vol - VOL_NUMBERS[0])

    # iss에 따른 월 매핑
    month_mapping = {
        1: "03",
        2: "06",
        3: "09",
        4: "12"
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

for file_path in sorted(csv_files):
    # 파일명에서 vol과 iss 추출
    filename = os.path.basename(file_path)
    match = re.search(r'vol(\d+)_iss(\d+)', filename)

    if match:
        vol_num = int(match.group(1))
        iss_num = int(match.group(2))

        # CSV 파일 읽기
        try:
            df = pd.read_csv(file_path)

            # date 칼럼 추가
            df['date'] = get_date(vol_num, iss_num)

            # affiliations 칼럼 추가 (저널명을 대문자로)
            df['affiliations'] = JOURNAL_PREFIX.upper()

            all_dfs.append(df)

            print(f"처리: {filename:30s} -> date: {df['date'].iloc[0]}, rows: {len(df)}")

        except Exception as e:
            print(f"❌ 오류 ({filename}): {e}")

print(f"\n{'='*60}")

# 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행")

    # 1. null 값 제거 (title, abstract 기준)
    print(f"\n[1단계] Null 값 제거 (title, abstract 기준)")
    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 2. 중복 제거 (title, abstract 기준)
    print(f"\n[2단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 3. 최종 컬럼 선택
    print(f"\n[3단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 4. 날짜별 통계
    print(f"\n[4단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 5. 결과 저장
    output_filename = f"{JOURNAL_PREFIX.upper()}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_PREFIX.upper()}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    print(final_df.head(3).to_string())

else:
    print("❌ 처리할 데이터가 없습니다.")

✓ 11개의 JIT 파일 발견

처리: JIT_vol38_iss1.csv             -> date: 2023-03, rows: 7
처리: JIT_vol38_iss2.csv             -> date: 2023-06, rows: 7
처리: JIT_vol38_iss3.csv             -> date: 2023-09, rows: 7
처리: JIT_vol38_iss4.csv             -> date: 2023-12, rows: 6
처리: JIT_vol39_iss1.csv             -> date: 2024-03, rows: 9
처리: JIT_vol39_iss2.csv             -> date: 2024-06, rows: 6
처리: JIT_vol39_iss3.csv             -> date: 2024-09, rows: 10
처리: JIT_vol39_iss4.csv             -> date: 2024-12, rows: 8
처리: JIT_vol40_iss1.csv             -> date: 2025-03, rows: 5
처리: JIT_vol40_iss2.csv             -> date: 2025-06, rows: 6
처리: JIT_vol40_iss3.csv             -> date: 2025-09, rows: 4

✓ 병합 완료: 총 75개 행

[1단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 75개 행
  - 제거 후: 50개 행
  - 제거됨: 25개 행

[2단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 50개 행
  - 제거 후: 50개 행
  - 제거됨: 0개 행

[3단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[4단계] 날짜별 통계
  - 2023-03: 6개
  - 2023-06:

## 6.JMIS.csv 만들기
- vol 40~42, iss 1~4로 2023년~2025년까지 4분기로 나누어서 date 생성
- 파일경로: /home/dslab/choi/Journal/Data/Academia/JMIS/JMIS_vol40_iss1.csv
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 JMIS로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [1]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "JMIS"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/JMIS/"
FILE_PATTERN = "JMIS_vol*_iss*.csv"
# vol-year 매핑
VOL_NUMBERS = [40, 41, 42]  # vol 40, 41, 42
START_YEAR = 2023  # vol 40 = 2023년
# ========================================

print(f"{'='*60}")
print(f"JMIS.csv 생성 시작")
print(f"{'='*60}\n")

# CSV 파일들 찾기
csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"⚠ '{FILE_PATTERN}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 JMIS 파일 발견\n")

# 날짜 매핑 함수
def get_date(vol, iss):
    """
    vol과 iss 번호로 날짜 생성
    vol 40 = 2023년, vol 41 = 2024년, vol 42 = 2025년
    iss 1~4 = 3월, 6월, 9월, 12월
    """
    if vol in VOL_NUMBERS:
        year = START_YEAR + (VOL_NUMBERS.index(vol))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (vol - VOL_NUMBERS[0])

    # iss에 따른 월 매핑 (4분기)
    month_mapping = {
        1: "03",  # 1분기
        2: "06",  # 2분기
        3: "09",  # 3분기
        4: "12"   # 4분기
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

print("[1단계] JMIS 파일 읽기 및 날짜 생성")
for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    # 파일명에서 vol과 iss 추출
    match = re.search(r'vol(\d+)_iss(\d+)', filename)

    if match:
        vol_num = int(match.group(1))
        iss_num = int(match.group(2))

        try:
            df = pd.read_csv(file_path)

            # date 칼럼 추가
            df['date'] = get_date(vol_num, iss_num)

            # affiliations 칼럼 추가
            df['affiliations'] = JOURNAL_NAME

            all_dfs.append(df)

            print(f"  - {filename:35s} -> date: {df['date'].iloc[0]}, rows: {len(df)}")

        except Exception as e:
            print(f"  ❌ 오류 ({filename}): {e}")
    else:
        print(f"  ⚠ 파일명 패턴 불일치: {filename}")

print(f"\n{'='*60}")

# 2. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 3. null 값 제거 (title, abstract 기준)
    print(f"\n[2단계] Null 값 제거 (title, abstract 기준)")

    # title과 abstract 컬럼 존재 여부 확인
    required_cols = ['title', 'abstract']
    missing_cols = [col for col in required_cols if col not in combined_df.columns]

    if missing_cols:
        print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
        print(f"  현재 컬럼: {list(combined_df.columns)}")
        exit()

    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 4. 중복 제거 (title, abstract 기준)
    print(f"\n[3단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 5. 최종 컬럼 선택
    print(f"\n[4단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 6. 날짜별 통계
    print(f"\n[5단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 7. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ Vol 범위: {min(VOL_NUMBERS)} - {max(VOL_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

    print(f"\n{'='*60}")
    print("✓ JMIS.csv 생성 완료!")
    print(f"{'='*60}")

else:
    print("❌ 처리할 데이터가 없습니다.")

JMIS.csv 생성 시작

✓ 11개의 JMIS 파일 발견

[1단계] JMIS 파일 읽기 및 날짜 생성
  - JMIS_vol40_iss1.csv                 -> date: 2023-03, rows: 12
  - JMIS_vol40_iss2.csv                 -> date: 2023-06, rows: 13
  - JMIS_vol40_iss3.csv                 -> date: 2023-09, rows: 12
  - JMIS_vol40_iss4.csv                 -> date: 2023-12, rows: 12
  - JMIS_vol41_iss1.csv                 -> date: 2024-03, rows: 12
  - JMIS_vol41_iss2.csv                 -> date: 2024-06, rows: 11
  - JMIS_vol41_iss3.csv                 -> date: 2024-09, rows: 11
  - JMIS_vol41_iss4.csv                 -> date: 2024-12, rows: 12
  - JMIS_vol42_iss1.csv                 -> date: 2025-03, rows: 11
  - JMIS_vol42_iss2.csv                 -> date: 2025-06, rows: 11
  - JMIS_vol42_iss3.csv                 -> date: 2025-09, rows: 11

✓ 병합 완료: 총 128개 행 (11개 파일)

[2단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 128개 행
  - 제거 후: 128개 행
  - 제거됨: 0개 행

[3단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 128개 행
  - 제거 후: 128개 행
  - 제거됨: 0개 행

[4단계] 

## 7.EJIS.csv 만들기
- vol 32 ~ 34, iss 1~6 -> 2,4,6,8,10,12월로 일단 저장 진행
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 JMIS로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [2]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "EJIS"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/EJIS"  # EJIS 파일들이 있는 디렉토리
FILE_PATTERN = "EJIS_vol*_iss*.csv"  # 또는 "EJIS_vol*_iss*.csv"
# vol-year 매핑 (예시: vol 32, 33, 34 = 2023, 2024, 2025)
VOL_NUMBERS = [32, 33, 34]  # 처리할 vol 번호 리스트 (필요시 수정)
START_YEAR = 2023  # 첫 번째 vol의 시작 연도
# ========================================

print(f"{'='*60}")
print(f"EJIS.csv 생성 시작")
print(f"{'='*60}\n")

# CSV 파일들 찾기
csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"⚠ '{FILE_PATTERN}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    print(f"다른 파일명 패턴이라면 코드 상단의 FILE_PATTERN을 수정하세요.")
    exit()

print(f"✓ {len(csv_files)}개의 EJIS 파일 발견\n")

# 날짜 매핑 함수
def get_date(vol, iss):
    """
    vol과 iss 번호로 날짜 생성
    iss 1-6을 2개월 단위로 균등 분배
    iss 1 → 2월 (1-2월)
    iss 2 → 4월 (3-4월)
    iss 3 → 6월 (5-6월)
    iss 4 → 8월 (7-8월)
    iss 5 → 10월 (9-10월)
    iss 6 → 12월 (11-12월)
    """
    if vol in VOL_NUMBERS:
        year = START_YEAR + (VOL_NUMBERS.index(vol))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (vol - VOL_NUMBERS[0])

    # iss에 따른 월 매핑 (2개월 단위 균등 분배)
    month_mapping = {
        1: "02",  # iss 1 → 1-2월
        2: "04",  # iss 2 → 3-4월
        3: "06",  # iss 3 → 5-6월
        4: "08",  # iss 4 → 7-8월
        5: "10",  # iss 5 → 9-10월
        6: "12"   # iss 6 → 11-12월
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

print("[1단계] EJIS 파일 읽기 및 날짜 생성")
for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    # 파일명에서 vol과 iss 추출
    match = re.search(r'vol(\d+)_iss(\d+)', filename)

    if match:
        vol_num = int(match.group(1))
        iss_num = int(match.group(2))

        try:
            df = pd.read_csv(file_path)

            # date 칼럼 추가
            df['date'] = get_date(vol_num, iss_num)

            # affiliations 칼럼 추가
            df['affiliations'] = JOURNAL_NAME

            all_dfs.append(df)

            print(f"  - {filename:35s} -> vol: {vol_num}, iss: {iss_num}, date: {df['date'].iloc[0]}, rows: {len(df)}")

        except Exception as e:
            print(f"  ❌ 오류 ({filename}): {e}")
    else:
        print(f"  ⚠ 파일명 패턴 불일치: {filename}")

print(f"\n{'='*60}")

# 2. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 3. null 값 제거 (title, abstract 기준)
    print(f"\n[2단계] Null 값 제거 (title, abstract 기준)")

    # title과 abstract 컬럼 존재 여부 확인
    required_cols = ['title', 'abstract']
    missing_cols = [col for col in required_cols if col not in combined_df.columns]

    if missing_cols:
        print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
        print(f"  현재 컬럼: {list(combined_df.columns)}")
        exit()

    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 4. 중복 제거 (title, abstract 기준)
    print(f"\n[3단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 5. 최종 컬럼 선택
    print(f"\n[4단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 6. 날짜별 통계
    print(f"\n[5단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 7. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ Vol 범위: {min(VOL_NUMBERS)} - {max(VOL_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")
    print(f"✓ Issue 매핑: iss 1→2월, iss 2→4월, iss 3→6월, iss 4→8월, iss 5→10월, iss 6→12월")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

    print(f"\n{'='*60}")
    print("✓ EJIS.csv 생성 완료!")
    print(f"{'='*60}")

else:
    print("❌ 처리할 데이터가 없습니다.")

EJIS.csv 생성 시작

✓ 18개의 EJIS 파일 발견

[1단계] EJIS 파일 읽기 및 날짜 생성
  - EJIS_vol32_iss1.csv                 -> vol: 32, iss: 1, date: 2023-02, rows: 8
  - EJIS_vol32_iss2.csv                 -> vol: 32, iss: 2, date: 2023-04, rows: 13
  - EJIS_vol32_iss3.csv                 -> vol: 32, iss: 3, date: 2023-06, rows: 13
  - EJIS_vol32_iss4.csv                 -> vol: 32, iss: 4, date: 2023-08, rows: 7
  - EJIS_vol32_iss5.csv                 -> vol: 32, iss: 5, date: 2023-10, rows: 8
  - EJIS_vol32_iss6.csv                 -> vol: 32, iss: 6, date: 2023-12, rows: 8
  - EJIS_vol33_iss1.csv                 -> vol: 33, iss: 1, date: 2024-02, rows: 5
  - EJIS_vol33_iss2.csv                 -> vol: 33, iss: 2, date: 2024-04, rows: 8
  - EJIS_vol33_iss3.csv                 -> vol: 33, iss: 3, date: 2024-06, rows: 7
  - EJIS_vol33_iss4.csv                 -> vol: 33, iss: 4, date: 2024-08, rows: 8
  - EJIS_vol33_iss5.csv                 -> vol: 33, iss: 5, date: 2024-10, rows: 9
  - EJIS_vol33_iss6.csv  

## 8.JAIS.csv 만들기
- vol 24 ~ 26, iss 1~6 -> 2,4,6,8,10,12월로 일단 저장 진행
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 JAIS로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [3]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "JAIS"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/JAIS"  # EJIS 파일들이 있는 디렉토리
FILE_PATTERN = "JAIS_vol*_iss*.csv"  # 또는 "EJIS_vol*_iss*.csv"
# vol-year 매핑 (예시: vol 32, 33, 34 = 2023, 2024, 2025)
VOL_NUMBERS = [24, 25, 26]  # 처리할 vol 번호 리스트 (필요시 수정)
START_YEAR = 2023  # 첫 번째 vol의 시작 연도
# ========================================

print(f"{'='*60}")
print(f"JAIS.csv 생성 시작")
print(f"{'='*60}\n")

# CSV 파일들 찾기
csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"⚠ '{FILE_PATTERN}' 패턴의 CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    print(f"다른 파일명 패턴이라면 코드 상단의 FILE_PATTERN을 수정하세요.")
    exit()

print(f"✓ {len(csv_files)}개의 EJIS 파일 발견\n")

# 날짜 매핑 함수
def get_date(vol, iss):
    """
    vol과 iss 번호로 날짜 생성
    iss 1-6을 2개월 단위로 균등 분배
    iss 1 → 2월 (1-2월)
    iss 2 → 4월 (3-4월)
    iss 3 → 6월 (5-6월)
    iss 4 → 8월 (7-8월)
    iss 5 → 10월 (9-10월)
    iss 6 → 12월 (11-12월)
    """
    if vol in VOL_NUMBERS:
        year = START_YEAR + (VOL_NUMBERS.index(vol))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (vol - VOL_NUMBERS[0])

    # iss에 따른 월 매핑 (2개월 단위 균등 분배)
    month_mapping = {
        1: "02",  # iss 1 → 1-2월
        2: "04",  # iss 2 → 3-4월
        3: "06",  # iss 3 → 5-6월
        4: "08",  # iss 4 → 7-8월
        5: "10",  # iss 5 → 9-10월
        6: "12"   # iss 6 → 11-12월
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

print("[1단계] EJIS 파일 읽기 및 날짜 생성")
for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    # 파일명에서 vol과 iss 추출
    match = re.search(r'vol(\d+)_iss(\d+)', filename)

    if match:
        vol_num = int(match.group(1))
        iss_num = int(match.group(2))

        try:
            df = pd.read_csv(file_path)

            # date 칼럼 추가
            df['date'] = get_date(vol_num, iss_num)

            # affiliations 칼럼 추가
            df['affiliations'] = JOURNAL_NAME

            all_dfs.append(df)

            print(f"  - {filename:35s} -> vol: {vol_num}, iss: {iss_num}, date: {df['date'].iloc[0]}, rows: {len(df)}")

        except Exception as e:
            print(f"  ❌ 오류 ({filename}): {e}")
    else:
        print(f"  ⚠ 파일명 패턴 불일치: {filename}")

print(f"\n{'='*60}")

# 2. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 3. null 값 제거 (title, abstract 기준)
    print(f"\n[2단계] Null 값 제거 (title, abstract 기준)")

    # title과 abstract 컬럼 존재 여부 확인
    required_cols = ['title', 'abstract']
    missing_cols = [col for col in required_cols if col not in combined_df.columns]

    if missing_cols:
        print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
        print(f"  현재 컬럼: {list(combined_df.columns)}")
        exit()

    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 4. 중복 제거 (title, abstract 기준)
    print(f"\n[3단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 5. 최종 컬럼 선택
    print(f"\n[4단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 6. 날짜별 통계
    print(f"\n[5단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 7. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ Vol 범위: {min(VOL_NUMBERS)} - {max(VOL_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(VOL_NUMBERS) - 1}")
    print(f"✓ Issue 매핑: iss 1→2월, iss 2→4월, iss 3→6월, iss 4→8월, iss 5→10월, iss 6→12월")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

    print(f"\n{'='*60}")
    print("✓ EJIS.csv 생성 완료!")
    print(f"{'='*60}")

else:
    print("❌ 처리할 데이터가 없습니다.")

JAIS.csv 생성 시작

✓ 17개의 EJIS 파일 발견

[1단계] EJIS 파일 읽기 및 날짜 생성
  - JAIS_vol24_iss1.csv                 -> vol: 24, iss: 1, date: 2023-02, rows: 10
  - JAIS_vol24_iss2.csv                 -> vol: 24, iss: 2, date: 2023-04, rows: 11
  - JAIS_vol24_iss3.csv                 -> vol: 24, iss: 3, date: 2023-06, rows: 10
  - JAIS_vol24_iss4.csv                 -> vol: 24, iss: 4, date: 2023-08, rows: 9
  - JAIS_vol24_iss5.csv                 -> vol: 24, iss: 5, date: 2023-10, rows: 11
  - JAIS_vol24_iss6.csv                 -> vol: 24, iss: 6, date: 2023-12, rows: 10
  - JAIS_vol25_iss1.csv                 -> vol: 25, iss: 1, date: 2024-02, rows: 15
  - JAIS_vol25_iss2.csv                 -> vol: 25, iss: 2, date: 2024-04, rows: 10
  - JAIS_vol25_iss3.csv                 -> vol: 25, iss: 3, date: 2024-06, rows: 11
  - JAIS_vol25_iss4.csv                 -> vol: 25, iss: 4, date: 2024-08, rows: 9
  - JAIS_vol25_iss5.csv                 -> vol: 25, iss: 5, date: 2024-10, rows: 10
  - JAIS_vol25_iss

## 9.JSIS.csv 만들기
- 데이터에 있는 vol 32 ~ 34, iss 1~4을 2023년 ~ 2025년, 3월, 6월, 9월, 12월로 나눠서 date 저장
- null 값 -> title, abstract 기준으로 제거
- 중복 제거 -> title, abstract 기준으로 제거
- affiliations 생성 -> 파일이름 참고해서 JSIS.csv로 저장
- 최종 컬럼 -> date, title, abstract, keywords, affiliations

In [5]:
import pandas as pd
import os
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "JSIS"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/JSIS"  # JSIS 파일들이 있는 디렉토리
FILE_PATTERN = "*.csv"  # JSIS 관련 csv 파일들
# volume-year 매핑
volume_NUMBERS = [32, 33, 34]  # volume 32, 33, 34
START_YEAR = 2023  # volume 32 = 2023년
# ========================================

print(f"{'='*60}")
print(f"JSIS.csv 생성 시작")
print(f"{'='*60}\n")

# CSV 파일들 찾기
csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"⚠ CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 CSV 파일 발견\n")

# 날짜 매핑 함수
def get_date(volume, issue):
    """
    volume과 issue 번호로 날짜 생성
    volume 32 = 2023년, volume 33 = 2024년, volume 34 = 2025년
    issue 1~4 = 3월, 6월, 9월, 12월
    """
    if pd.isna(volume) or pd.isna(issue):
        return None

    volume = int(volume)
    issue = int(issue)

    if volume in volume_NUMBERS:
        year = START_YEAR + (volume_NUMBERS.index(volume))
    else:
        # volume_NUMBERS에 없는 경우, 첫 번째 volume을 기준으로 계산
        year = START_YEAR + (volume - volume_NUMBERS[0])

    # issue에 따른 월 매핑 (4분기)
    month_mapping = {
        1: "03",  # 1분기
        2: "06",  # 2분기
        3: "09",  # 3분기
        4: "12"   # 4분기
    }

    month = month_mapping.get(issue, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

print("[1단계] JSIS 파일 읽기")
for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    try:
        df = pd.read_csv(file_path)

        # volume과 issue 컬럼이 있는지 확인
        if 'volume' in df.columns and 'issue' in df.columns:
            all_dfs.append(df)
            print(f"  ✓ {filename:40s} -> {len(df)}개 행 (volume, issue 컬럼 있음)")
        else:
            print(f"  - {filename:40s} -> volume/issue 컬럼 없음 (건너뜀)")

    except Exception as e:
        print(f"  ❌ 오류 ({filename}): {e}")

print(f"\n{'='*60}")

# 2. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 3. date 칼럼 생성
    print(f"\n[2단계] date 칼럼 생성 (volume, issue 기반)")
    combined_df['date'] = combined_df.apply(lambda row: get_date(row['volume'], row['issue']), axis=1)

    # date 생성 통계
    date_created = combined_df['date'].notna().sum()
    date_null = combined_df['date'].isna().sum()
    print(f"  - date 생성 완료: {date_created}개")
    if date_null > 0:
        print(f"  ⚠ date 생성 실패: {date_null}개 (volume/issue 값이 없음)")

    # 4. affiliations 칼럼 생성
    print(f"\n[3단계] affiliations 생성")
    combined_df['affiliations'] = JOURNAL_NAME
    print(f"  - affiliations: {JOURNAL_NAME}")

    # 5. null 값 제거 (title, abstract 기준)
    print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

    # title과 abstract 컬럼 존재 여부 확인
    required_cols = ['title', 'abstract']
    missing_cols = [col for col in required_cols if col not in combined_df.columns]

    if missing_cols:
        print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
        print(f"  현재 컬럼: {list(combined_df.columns)}")
        exit()

    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 6. 중복 제거 (title, abstract 기준)
    print(f"\n[5단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 7. 최종 컬럼 선택
    print(f"\n[6단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 8. 날짜별 통계
    print(f"\n[7단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 9. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ volume 범위: {min(volume_NUMBERS)} - {max(volume_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(volume_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

    print(f"\n{'='*60}")
    print("✓ JSIS.csv 생성 완료!")
    print(f"{'='*60}")

else:
    print("❌ volume, issue 컬럼이 있는 CSV 파일을 찾을 수 없습니다.")

JSIS.csv 생성 시작

✓ 8개의 CSV 파일 발견

[1단계] JSIS 파일 읽기
  ✓ JSIS_vol32_iss1to4.csv                   -> 28개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol32to32_iss1to4.csv               -> 28개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol33_iss1to4.csv                   -> 30개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol33to33_iss1to4.csv               -> 30개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol34_iss1to2.csv                   -> 21개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol34_iss3to4.csv                   -> 11개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol34to34_iss1to2.csv               -> 21개 행 (volume, issue 컬럼 있음)
  ✓ JSIS_vol34to34_iss3to4.csv               -> 11개 행 (volume, issue 컬럼 있음)

✓ 병합 완료: 총 180개 행 (8개 파일)

[2단계] date 칼럼 생성 (volume, issue 기반)
  - date 생성 완료: 180개

[3단계] affiliations 생성
  - affiliations: JSIS

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 180개 행
  - 제거 후: 130개 행
  - 제거됨: 50개 행

[5단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 130개 행
  - 제거 후: 65개 행
  - 제거됨: 65개 행

[6단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title

## 10.ISJ.csv 만들기
- vol, iss 데이터에 있는 거로 date 만들기
- 나머지 동일하게 진행.

In [3]:
import pandas as pd
import os
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "ISJ"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/ISJ"  # ISJ 파일들이 있는 디렉토리
FILE_PATTERN = "*.csv"  # ISJ 관련 csv 파일들
# volume-year 매핑
volume_NUMBERS = [33, 34, 35] # volume 32, 33, 34
START_YEAR = 2023  # volume 32 = 2023년
# ========================================

print(f"{'='*60}")
print(f"ISJ.csv 생성 시작")
print(f"{'='*60}\n")

# CSV 파일들 찾기
csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"⚠ CSV 파일을 찾을 수 없습니다.")
    print(f"경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 CSV 파일 발견\n")

# 날짜 매핑 함수
def get_date(volume, iss):
    """
    vol과 iss 번호로 날짜 생성
    iss 1-6을 2개월 단위로 균등 분배
    iss 1 → 2월 (1-2월)
    iss 2 → 4월 (3-4월)
    iss 3 → 6월 (5-6월)
    iss 4 → 8월 (7-8월)
    iss 5 → 10월 (9-10월)
    iss 6 → 12월 (11-12월)
    """
    if volume in volume_NUMBERS:
        year = START_YEAR + (volume_NUMBERS.index(volume))
    else:
        # VOL_NUMBERS에 없는 경우, 첫 번째 vol을 기준으로 계산
        year = START_YEAR + (volume - volume_NUMBERS[0])

    # iss에 따른 월 매핑 (2개월 단위 균등 분배)
    month_mapping = {
        1: "02",  # iss 1 → 1-2월
        2: "04",  # iss 2 → 3-4월
        3: "06",  # iss 3 → 5-6월
        4: "08",  # iss 4 → 7-8월
        5: "10",  # iss 5 → 9-10월
        6: "12"   # iss 6 → 11-12월
    }

    month = month_mapping.get(iss, "03")
    return f"{year}-{month}"

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

print("[1단계] ISJ 파일 읽기")
for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)

    try:
        df = pd.read_csv(file_path)

        # volume과 issue 컬럼이 있는지 확인
        if 'volume' in df.columns and 'issue' in df.columns:
            all_dfs.append(df)
            print(f"  ✓ {filename:40s} -> {len(df)}개 행 (volume, issue 컬럼 있음)")
        else:
            print(f"  - {filename:40s} -> volume/issue 컬럼 없음 (건너뜀)")

    except Exception as e:
        print(f"  ❌ 오류 ({filename}): {e}")

print(f"\n{'='*60}")

# 2. 모든 데이터프레임 합치기
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print(f"✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

    # 3. date 칼럼 생성
    print(f"\n[2단계] date 칼럼 생성 (volume, issue 기반)")
    combined_df['date'] = combined_df.apply(lambda row: get_date(row['volume'], row['issue']), axis=1)

    # date 생성 통계
    date_created = combined_df['date'].notna().sum()
    date_null = combined_df['date'].isna().sum()
    print(f"  - date 생성 완료: {date_created}개")
    if date_null > 0:
        print(f"  ⚠ date 생성 실패: {date_null}개 (volume/issue 값이 없음)")

    # 4. affiliations 칼럼 생성
    print(f"\n[3단계] affiliations 생성")
    combined_df['affiliations'] = JOURNAL_NAME
    print(f"  - affiliations: {JOURNAL_NAME}")

    # 5. null 값 제거 (title, abstract 기준)
    print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

    # title과 abstract 컬럼 존재 여부 확인
    required_cols = ['title', 'abstract']
    missing_cols = [col for col in required_cols if col not in combined_df.columns]

    if missing_cols:
        print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
        print(f"  현재 컬럼: {list(combined_df.columns)}")
        exit()

    before_null = len(combined_df)
    combined_df = combined_df.dropna(subset=['title', 'abstract'])
    after_null = len(combined_df)
    print(f"  - 제거 전: {before_null}개 행")
    print(f"  - 제거 후: {after_null}개 행")
    print(f"  - 제거됨: {before_null - after_null}개 행")

    # 6. 중복 제거 (title, abstract 기준)
    print(f"\n[5단계] 중복 제거 (title, abstract 기준)")
    before_dup = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
    after_dup = len(combined_df)
    print(f"  - 제거 전: {before_dup}개 행")
    print(f"  - 제거 후: {after_dup}개 행")
    print(f"  - 제거됨: {before_dup - after_dup}개 행")

    # 7. 최종 컬럼 선택
    print(f"\n[6단계] 최종 컬럼 선택")
    required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in combined_df.columns]
    missing_columns = [col for col in required_columns if col not in combined_df.columns]

    if missing_columns:
        print(f"  ⚠ 누락된 컬럼: {missing_columns}")
        for col in missing_columns:
            combined_df[col] = None

    final_df = combined_df[required_columns].copy()
    print(f"  - 최종 컬럼: {list(final_df.columns)}")

    # 8. 날짜별 통계
    print(f"\n[7단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()
    for date, count in date_stats.items():
        print(f"  - {date}: {count}개")

    # 9. 결과 저장
    output_filename = f"{JOURNAL_NAME}.csv"
    output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia", output_filename)
    final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n{'='*60}")
    print(f"✓ 최종 저장 완료!")
    print(f"✓ 파일 경로: {output_path}")
    print(f"✓ 최종 행 수: {len(final_df)}개")
    print(f"✓ 컬럼: {list(final_df.columns)}")
    print(f"✓ 저널: {JOURNAL_NAME}")
    print(f"✓ volume 범위: {min(volume_NUMBERS)} - {max(volume_NUMBERS)}")
    print(f"✓ 연도 범위: {START_YEAR} - {START_YEAR + len(volume_NUMBERS) - 1}")

    # 샘플 데이터 미리보기
    print(f"\n[데이터 미리보기]")
    if len(final_df) > 0:
        print(final_df.head(3).to_string())
    else:
        print("  - 데이터가 없습니다.")

    print(f"\n{'='*60}")
    print("✓ ISJ.csv 생성 완료!")
    print(f"{'='*60}")

else:
    print("❌ volume, issue 컬럼이 있는 CSV 파일을 찾을 수 없습니다.")

ISJ.csv 생성 시작

✓ 1개의 CSV 파일 발견

[1단계] ISJ 파일 읽기
  ✓ ISJ.csv                                  -> 180개 행 (volume, issue 컬럼 있음)

✓ 병합 완료: 총 180개 행 (1개 파일)

[2단계] date 칼럼 생성 (volume, issue 기반)
  - date 생성 완료: 180개

[3단계] affiliations 생성
  - affiliations: ISJ

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 180개 행
  - 제거 후: 148개 행
  - 제거됨: 32개 행

[5단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 148개 행
  - 제거 후: 148개 행
  - 제거됨: 0개 행

[6단계] 최종 컬럼 선택
  ⚠ 누락된 컬럼: ['keywords']
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[7단계] 날짜별 통계
  - 2023-08: 6개
  - 2023-10: 7개
  - 2023-12: 6개
  - 2024-02: 8개
  - 2024-04: 8개
  - 2024-06: 9개
  - 2024-08: 12개
  - 2024-10: 11개
  - 2024-12: 10개
  - 2025-02: 11개
  - 2025-04: 11개
  - 2025-06: 8개
  - 2025-08: 7개
  - 2025-10: 9개
  - nan-03: 25개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Academia/ISJ.csv
✓ 최종 행 수: 148개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 저널: ISJ
✓ volume 범위: 33 - 35
✓ 연도 범위: 2023 - 2025

[데이터 미

### 10.1.ISJ 아직 게시 안된 것들 어떻게 해야 할까...
- 빼는 쪽으로..

## 11. ICIS.csv 만들기
- date 컬럼 그대로 두기
- affiliations 칼럼 생성 -> ICIS로 변경
- null, 중복 제거 -> title, abstract 기준
- 2023년 ~ 2025년만 남기기
- 최종 데이터 저장 -> ICIS.csv로 필요한 컬럼만 저장 -> date, title, abstract, keywords, affiliations

In [2]:
import pandas as pd
import os

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "ICIS"
INPUT_FILE_PATH = "/home/dslab/choi/Journal/Data/Academia/ICIS,HICSS/ICIS.csv"  # 또는 다른 경로
# ========================================

print(f"{'='*60}")
print(f"ICIS.csv 생성 시작")
print(f"{'='*60}\n")

# 1. ICIS 데이터 읽기
print("[1단계] ICIS 데이터 읽기")

if not os.path.exists(INPUT_FILE_PATH):
    print(f"  ❌ 파일이 없습니다: {INPUT_FILE_PATH}")
    print(f"  ⚠ 다른 경로에 있다면 코드 상단의 INPUT_FILE_PATH를 수정하세요.")
    exit()

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"  ✓ 파일 읽기 완료: {INPUT_FILE_PATH}")
    print(f"    - 행 수: {len(df)}")
    print(f"    - 컬럼: {list(df.columns)}")
except Exception as e:
    print(f"  ❌ 파일 읽기 오류: {e}")
    exit()

print(f"\n{'='*60}")

# 2. date 컬럼 확인
print(f"\n[2단계] date 컬럼 확인")

if 'date' in df.columns:
    print(f"  ✓ date 컬럼이 존재합니다")
    print(f"  - date는 그대로 유지됩니다")
    if len(df) > 0:
        print(f"  - 샘플 date: {df['date'].iloc[0]}")
else:
    print(f"  ⚠ date 컬럼이 없습니다")
    df['date'] = None

# 3. affiliations 칼럼 생성
print(f"\n[3단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 4. null 값 제거 (title, abstract 기준)
print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

# 필수 컬럼 존재 여부 확인
required_cols = ['title', 'abstract']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 5. 연도 필터링 (2023-2025)
print(f"\n[5단계] 연도 필터링 (2023-2025)")

import re

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

VALID_YEARS = [2023, 2024, 2025]
before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 6. 중복 제거 (title, abstract 기준)
print(f"\n[6단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 7. 최종 컬럼 선택
print(f"\n[7단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 7. 날짜별 통계 (date가 있는 경우)
if 'date' in final_df.columns and final_df['date'].notna().any():
    print(f"\n[8단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"\n[8단계] 날짜 정보 없음")

# 9. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia/", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 저널: {JOURNAL_NAME}")
print(f"✓ 연도 범위: 2023 - 2025")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ ICIS.csv 생성 완료!")
print(f"{'='*60}")

ICIS.csv 생성 시작

[1단계] ICIS 데이터 읽기
  ✓ 파일 읽기 완료: /home/dslab/choi/Journal/Data/Academia/ICIS,HICSS/ICIS.csv
    - 행 수: 1548
    - 컬럼: ['title', 'date', 'abstract', 'keywords', 'authors', 'affiliations']


[2단계] date 컬럼 확인
  ✓ date 컬럼이 존재합니다
  - date는 그대로 유지됩니다
  - 샘플 date: 2021

[3단계] affiliations 생성
  - affiliations: ICIS

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 1548개 행
  - 제거 후: 1528개 행
  - 제거됨: 20개 행

[5단계] 연도 필터링 (2023-2025)
  - 필터링 전: 1528개 행
  - 필터링 후: 836개 행
  - 제거됨: 692개 행

[6단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 836개 행
  - 제거 후: 835개 행
  - 제거됨: 1개 행

[7단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[8단계] 날짜별 통계
  - 2023: 391개
  - 2024: 444개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Academia/ICIS.csv
✓ 최종 행 수: 835개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 저널: ICIS
✓ 연도 범위: 2023 - 2025

[데이터 미리보기]
     date                                                                                               

## 12. HICSS.csv 만들기
- date 컬럼 그대로 두기
- affiliations 칼럼 생성 -> HICSS로 변경
- null, 중복 제거 -> title, abstract 기준
- 2023년 ~ 2025년만 남기기
- 최종 데이터 저장 -> HICSS.csv로 필요한 컬럼만 저장 -> date, title, abstract, keywords, affiliations

In [3]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "HICSS"
INPUT_FILE_PATH = "/home/dslab/choi/Journal/Data/Academia/ICIS,HICSS/HICSS.csv"  # 또는 다른 경로
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"HICSS.csv 생성 시작")
print(f"{'='*60}\n")

# 1. HICSS 데이터 읽기
print("[1단계] HICSS 데이터 읽기")

if not os.path.exists(INPUT_FILE_PATH):
    print(f"  ❌ 파일이 없습니다: {INPUT_FILE_PATH}")
    print(f"  ⚠ 다른 경로에 있다면 코드 상단의 INPUT_FILE_PATH를 수정하세요.")
    exit()

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"  ✓ 파일 읽기 완료: {INPUT_FILE_PATH}")
    print(f"    - 행 수: {len(df)}")
    print(f"    - 컬럼: {list(df.columns)}")
except Exception as e:
    print(f"  ❌ 파일 읽기 오류: {e}")
    exit()

print(f"\n{'='*60}")

# 2. date 컬럼 확인
print(f"\n[2단계] date 컬럼 확인")

if 'date' in df.columns:
    print(f"  ✓ date 컬럼이 존재합니다")
    print(f"  - date는 그대로 유지됩니다")
    if len(df) > 0:
        print(f"  - 샘플 date: {df['date'].iloc[0]}")
else:
    print(f"  ⚠ date 컬럼이 없습니다")
    df['date'] = None

# 3. affiliations 칼럼 생성
print(f"\n[3단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 4. null 값 제거 (title, abstract 기준)
print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

# 필수 컬럼 존재 여부 확인
required_cols = ['title', 'abstract']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"  ❌ 필수 컬럼이 없습니다: {missing_cols}")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 5. 연도 필터링 (2023-2025)
print(f"\n[5단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 6. 중복 제거 (title, abstract 기준)
print(f"\n[6단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 7. 최종 컬럼 선택
print(f"\n[7단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 8. 날짜별 통계 (date가 있는 경우)
if 'date' in final_df.columns and final_df['date'].notna().any():
    print(f"\n[8단계] 날짜별 통계")
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"\n[8단계] 날짜 정보 없음")

# 9. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Academia/", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 저널: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ HICSS.csv 생성 완료!")
print(f"{'='*60}")

HICSS.csv 생성 시작

[1단계] HICSS 데이터 읽기
  ✓ 파일 읽기 완료: /home/dslab/choi/Journal/Data/Academia/ICIS,HICSS/HICSS.csv
    - 행 수: 3506
    - 컬럼: ['title', 'date', 'abstract', 'keywords', 'authors']


[2단계] date 컬럼 확인
  ✓ date 컬럼이 존재합니다
  - date는 그대로 유지됩니다
  - 샘플 date: 2021-01-05

[3단계] affiliations 생성
  - affiliations: HICSS

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 3506개 행
  - 제거 후: 3506개 행
  - 제거됨: 0개 행

[5단계] 연도 필터링 (2023-2025)
  - 필터링 전: 3506개 행
  - 필터링 후: 2098개 행
  - 제거됨: 1408개 행

[6단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 2098개 행
  - 제거 후: 2098개 행
  - 제거됨: 0개 행

[7단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[8단계] 날짜별 통계
  - 2023-01-03: 664개
  - 2024-01-03: 726개
  - 2025-01-07: 708개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Academia/HICSS.csv
✓ 최종 행 수: 2098개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 저널: HICSS
✓ 연도 범위: 2023 - 2025

[데이터 미리보기]
            date                                                       

# Industry

## 1. TheVerge.csv 만들기
- date 컬럼의 시간을 달까지만 남기기
- affiliations 칼럼 생성 -> TheVerge.csv로 저장
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 TheVerge.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations

In [5]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "TheVerge"
INPUT_FILE_PATH = "/home/dslab/choi/Journal/Data/Industry/TheVerge/TheVerge.csv"  # 또는 다른 파일명
# ========================================

print(f"{'='*60}")
print(f"TheVerge.csv 생성 시작")
print(f"{'='*60}\n")

# 1. TheVerge 데이터 읽기
print("[1단계] TheVerge 데이터 읽기")

if not os.path.exists(INPUT_FILE_PATH):
    print(f"  ❌ 파일이 없습니다: {INPUT_FILE_PATH}")
    print(f"  ⚠ 다른 경로에 있다면 코드 상단의 INPUT_FILE_PATH를 수정하세요.")
    exit()

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"  ✓ 파일 읽기 완료: {INPUT_FILE_PATH}")
    print(f"    - 행 수: {len(df)}")
    print(f"    - 컬럼: {list(df.columns)}")
except Exception as e:
    print(f"  ❌ 파일 읽기 오류: {e}")
    exit()

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (시간을 달까지만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원: YYYY-MM-DD, YYYY-MM-DD HH:MM:SS, YYYY/MM/DD 등
    """
    if pd.isna(date_str):
        return None

    date_str = str(date_str)

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD, YYYY-MM-DD HH:MM:SS 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. content를 abstract로 매핑
print(f"\n[3단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
else:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None

# 4. affiliations 칼럼 생성
print(f"\n[4단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin([2023, 2024, 2025])]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: 2023 - 2025")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ TheVerge.csv 생성 완료!")
print(f"{'='*60}")

TheVerge.csv 생성 시작

[1단계] TheVerge 데이터 읽기
  ✓ 파일 읽기 완료: /home/dslab/choi/Journal/Data/Industry/TheVerge/TheVerge.csv
    - 행 수: 4712
    - 컬럼: ['date', 'title', 'content', 'keywords', 'url']


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: 2025-01-20T22:00:00+09:00 → 2025-01
  - 변환 완료: 4710개
  ⚠ 변환 실패: 2개

[3단계] content → abstract 매핑
  ✓ content 컬럼을 abstract로 복사

[4단계] affiliations 생성
  - affiliations: TheVerge

[5단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 4712개 행
  - 제거 후: 4710개 행
  - 제거됨: 2개 행

[6단계] 연도 필터링 (2023-2025)
  - 필터링 전: 4710개 행
  - 필터링 후: 4597개 행
  - 제거됨: 113개 행

[7단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 4597개 행
  - 제거 후: 4584개 행
  - 제거됨: 13개 행

[8단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[9단계] 날짜별 통계
  - 총 34개의 날짜
  - 첫 날짜: 2023-01 (50개)
  - 마지막 날짜: 2025-10 (123개)

  [최근 10개 날짜]
    - 2025-01: 138개
    - 2025-02: 124개
    - 2025-03: 107개
    - 2025-04: 146개
    - 2025-05: 159개
    - 2025-06: 131개
    - 2025-07: 138개
    - 2025-08:

## 2. TechCrunch.csv 만들기
- date 컬럼의 시간을 달까지만 남기기
- affiliations 칼럼 생성 -> TechCrunch.csv로 저장
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 TechCrunch.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [6]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "TechCrunch"
INPUT_FILE_PATH = "/home/dslab/choi/Journal/Data/Industry/TechCrunch/TechCrunch.csv"  # 또는 다른 파일명
# ========================================

print(f"{'='*60}")
print(f"TechCrunch.csv 생성 시작")
print(f"{'='*60}\n")

# 1. TechCrunch 데이터 읽기
print("[1단계] TechCrunch 데이터 읽기")

if not os.path.exists(INPUT_FILE_PATH):
    print(f"  ❌ 파일이 없습니다: {INPUT_FILE_PATH}")
    print(f"  ⚠ 다른 경로에 있다면 코드 상단의 INPUT_FILE_PATH를 수정하세요.")
    exit()

try:
    df = pd.read_csv(INPUT_FILE_PATH)
    print(f"  ✓ 파일 읽기 완료: {INPUT_FILE_PATH}")
    print(f"    - 행 수: {len(df)}")
    print(f"    - 컬럼: {list(df.columns)}")
except Exception as e:
    print(f"  ❌ 파일 읽기 오류: {e}")
    exit()

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (시간을 달까지만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원: YYYY-MM-DD, YYYY-MM-DD HH:MM:SS, YYYY/MM/DD 등
    """
    if pd.isna(date_str):
        return None

    date_str = str(date_str)

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD, YYYY-MM-DD HH:MM:SS 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. content를 abstract로 매핑
print(f"\n[3단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
else:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None

# 4. affiliations 칼럼 생성
print(f"\n[4단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin([2023, 2024, 2025])]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: 2023 - 2025")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ TechCrunch.csv 생성 완료!")
print(f"{'='*60}")

TechCrunch.csv 생성 시작

[1단계] TechCrunch 데이터 읽기
  ✓ 파일 읽기 완료: /home/dslab/choi/Journal/Data/Industry/TechCrunch/TechCrunch.csv
    - 행 수: 727
    - 컬럼: ['title', 'date', 'content', 'keywords', 'url']


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: 2025-10-20T01:15:50-07:00 → 2025-10
  - 변환 완료: 717개
  ⚠ 변환 실패: 10개

[3단계] content → abstract 매핑
  ✓ content 컬럼을 abstract로 복사

[4단계] affiliations 생성
  - affiliations: TechCrunch

[5단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 727개 행
  - 제거 후: 727개 행
  - 제거됨: 0개 행

[6단계] 연도 필터링 (2023-2025)
  - 필터링 전: 727개 행
  - 필터링 후: 717개 행
  - 제거됨: 10개 행

[7단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 717개 행
  - 제거 후: 715개 행
  - 제거됨: 2개 행

[8단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[9단계] 날짜별 통계
  - 2025-04: 119개
  - 2025-05: 134개
  - 2025-06: 115개
  - 2025-07: 109개
  - 2025-08: 70개
  - 2025-09: 95개
  - 2025-10: 73개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Industry/TechCrunch.csv
✓ 최종 행 수: 715개
✓ 컬럼: ['date', 'tit

## 3.TheGuardian.csv 만들기
- 폴더에 있는 TheGuardian_*.csv를 다 병합
- date 컬럼의 시간을 달까지만 남기기
- affiliations 칼럼 생성 -> TheGuardian.csv로 저장
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 TechCrunch.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [4]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "TheGuardian"
DATA_DIR = "/home/dslab/choi/Journal/Data/Industry/TheGuardian"  # TheGuardian 파일들이 있는 디렉토리
FILE_PATTERN = "TheGuardian_*.csv"  # TheGuardian_로 시작하는 모든 csv 파일
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"TheGuardian.csv 생성 시작")
print(f"{'='*60}\n")

# 1. TheGuardian 데이터 읽기 및 병합
print("[1단계] TheGuardian 파일들 읽기 및 병합")

csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"  ❌ '{FILE_PATTERN}' 패턴의 파일을 찾을 수 없습니다.")
    print(f"  경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 TheGuardian 파일 발견\n")

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)
    try:
        temp_df = pd.read_csv(file_path)
        all_dfs.append(temp_df)
        print(f"  - {filename:40s} -> {len(temp_df)}개 행")
    except Exception as e:
        print(f"  ❌ 오류 ({filename}): {e}")

if not all_dfs:
    print("  ❌ 읽을 수 있는 파일이 없습니다.")
    exit()

# 모든 데이터프레임 병합
df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (시간을 달까지만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원:
    - Sat 4 Nov 2023 16.00 GMT
    - Mon 20 Oct 2025 18.40 BST
    - 1 August 2024
    - YYYY-MM-DD, YYYY-MM-DD HH:MM:SS, YYYY/MM/DD 등
    """
    if pd.isna(date_str):
        return None

    date_str = str(date_str)

    # 월 이름 매핑
    month_names = {
        'January': '01', 'Jan': '01',
        'February': '02', 'Feb': '02',
        'March': '03', 'Mar': '03',
        'April': '04', 'Apr': '04',
        'May': '05',
        'June': '06', 'Jun': '06',
        'July': '07', 'Jul': '07',
        'August': '08', 'Aug': '08',
        'September': '09', 'Sep': '09',
        'October': '10', 'Oct': '10',
        'November': '11', 'Nov': '11',
        'December': '12', 'Dec': '12'
    }

    # "Day DD Mon YYYY" 또는 "DD Month YYYY" 패턴 처리
    for month_name, month_num in month_names.items():
        if month_name in date_str:
            # 연도 추출 (4자리 숫자)
            year_match = re.search(r'\b(\d{4})\b', date_str)
            if year_match:
                year = year_match.group(1)
                return f"{year}-{month_num}"

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD, YYYY-MM-DD HH:MM:SS 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. content를 abstract로 매핑
print(f"\n[3단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
else:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None

# 4. affiliations 칼럼 생성
print(f"\n[4단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ TheGuardian.csv 생성 완료!")
print(f"{'='*60}")

TheGuardian.csv 생성 시작

[1단계] TheGuardian 파일들 읽기 및 병합
✓ 27개의 TheGuardian 파일 발견

  - TheGuardian_10.csv                       -> 197개 행
  - TheGuardian_100.csv                      -> 91개 행
  - TheGuardian_105.csv                      -> 189개 행
  - TheGuardian_110.csv                      -> 19개 행
  - TheGuardian_115.csv                      -> 117개 행
  - TheGuardian_120.csv                      -> 215개 행
  - TheGuardian_125.csv                      -> 312개 행
  - TheGuardian_130.csv                      -> 98개 행
  - TheGuardian_131.csv                      -> 19개 행
  - TheGuardian_15.csv                       -> 295개 행
  - TheGuardian_20.csv                       -> 395개 행
  - TheGuardian_25.csv                       -> 494개 행
  - TheGuardian_30.csv                       -> 592개 행
  - TheGuardian_35.csv                       -> 691개 행
  - TheGuardian_40.csv                       -> 95개 행
  - TheGuardian_45.csv                       -> 19개 행
  - TheGuardian_5.csv                        ->

## 4. BBC.csv 만들기
- BBC_*.csv를 다 병합
- date 컬럼의 시간을 달까지만 남기기
- affiliations 칼럼 생성 -> BBC.csv로 저장
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 BBC.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [3]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "BBC"
DATA_DIR = "/home/dslab/choi/Journal/Data/Industry/BBC"  # BBC 파일들이 있는 디렉토리
FILE_PATTERN = "BBC_*.csv"  # BBC_로 시작하는 모든 csv 파일
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"BBC.csv 생성 시작")
print(f"{'='*60}\n")

# 1. BBC 데이터 읽기 및 병합
print("[1단계] BBC 파일들 읽기 및 병합")

csv_files = glob(os.path.join(DATA_DIR, FILE_PATTERN))

if not csv_files:
    print(f"  ❌ '{FILE_PATTERN}' 패턴의 파일을 찾을 수 없습니다.")
    print(f"  경로 확인: {DATA_DIR}")
    exit()

print(f"✓ {len(csv_files)}개의 BBC 파일 발견\n")

# 모든 데이터프레임을 저장할 리스트
all_dfs = []

for file_path in sorted(csv_files):
    filename = os.path.basename(file_path)
    try:
        temp_df = pd.read_csv(file_path)
        all_dfs.append(temp_df)
        print(f"  - {filename:40s} -> {len(temp_df)}개 행")
    except Exception as e:
        print(f"  ❌ 오류 ({filename}): {e}")

if not all_dfs:
    print("  ❌ 읽을 수 있는 파일이 없습니다.")
    exit()

# 모든 데이터프레임 병합
df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (시간을 달까지만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원:
    - 1 August 2024
    - 20 Jan 2023
    - Mon 20 Oct 2025 18.40 BST
    - YYYY-MM-DD, YYYY-MM-DD HH:MM:SS
    - YYYY/MM/DD 등
    """
    if pd.isna(date_str):
        return None

    date_str = str(date_str)

    # 월 이름 매핑
    month_names = {
        'January': '01', 'Jan': '01',
        'February': '02', 'Feb': '02',
        'March': '03', 'Mar': '03',
        'April': '04', 'Apr': '04',
        'May': '05',
        'June': '06', 'Jun': '06',
        'July': '07', 'Jul': '07',
        'August': '08', 'Aug': '08',
        'September': '09', 'Sep': '09',
        'October': '10', 'Oct': '10',
        'November': '11', 'Nov': '11',
        'December': '12', 'Dec': '12'
    }

    # "DD Month YYYY" 또는 "Day DD Mon YYYY" 패턴 처리
    for month_name, month_num in month_names.items():
        if month_name in date_str:
            # 연도 추출 (4자리 숫자)
            year_match = re.search(r'\b(\d{4})\b', date_str)
            if year_match:
                year = year_match.group(1)
                return f"{year}-{month_num}"

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD, YYYY-MM-DD HH:MM:SS 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. content를 abstract로 매핑
print(f"\n[3단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
else:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None

# 4. abstract 정제 (오류 메시지를 null로 변환)
print(f"\n[4단계] abstract 정제 (오류 메시지 제거)")

# "내용 추출 실패" 같은 텍스트를 null로 변환
error_keywords = ['내용 추출 실패', '추출 실패', 'extraction failed', 'failed', 'error']

def clean_abstract(text):
    """abstract에서 오류 메시지를 null로 변환"""
    if pd.isna(text):
        return None

    text_lower = str(text).lower().strip()

    # 빈 문자열
    if not text_lower or text_lower == '':
        return None

    # 오류 키워드 체크
    for keyword in error_keywords:
        if keyword in text_lower:
            return None

    # 너무 짧은 텍스트 (5자 이하)
    if len(text_lower) <= 5:
        return None

    return text

before_clean = df['abstract'].notna().sum()
df['abstract'] = df['abstract'].apply(clean_abstract)
after_clean = df['abstract'].notna().sum()

print(f"  - 정제 전: {before_clean}개")
print(f"  - 정제 후: {after_clean}개")
print(f"  - 제거됨: {before_clean - after_clean}개 (오류 메시지 포함)")

# 5. affiliations 칼럼 생성
print(f"\n[5단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 6. null 값 제거 (title, content 기준)
print(f"\n[6단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 7. 연도 필터링 (2023-2025)
print(f"\n[7단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 8. 중복 제거 (title, content 기준)
print(f"\n[8단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 9. 최종 컬럼 선택
print(f"\n[9단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 10. 날짜별 통계
print(f"\n[10단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 11. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ BBC.csv 생성 완료!")
print(f"{'='*60}")

BBC.csv 생성 시작

[1단계] BBC 파일들 읽기 및 병합
✓ 15개의 BBC 파일 발견

  - BBC_100.csv                              -> 9개 행
  - BBC_121.csv                              -> 188개 행
  - BBC_139.csv                              -> 159개 행
  - BBC_20.csv                               -> 155개 행
  - BBC_202.csv                              -> 462개 행
  - BBC_27.csv                               -> 56개 행
  - BBC_30.csv                               -> 22개 행
  - BBC_40.csv                               -> 126개 행
  - BBC_41.csv                               -> 179개 행
  - BBC_50.csv                               -> 80개 행
  - BBC_60.csv                               -> 9개 행
  - BBC_61.csv                               -> 86개 행
  - BBC_80.csv                               -> 97개 행
  - BBC_81.csv                               -> 9개 행
  - BBC_90.csv                               -> 163개 행

✓ 병합 완료: 총 1800개 행 (15개 파일)


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: 19 June 2017 → 2017-06
  - 변환 완료: 1744개
  ⚠ 변환 실패: 56개

## 5.WallStreetJournal.csv 만들기
- 데이터 병합
- date 형식이 지금 23 Aug 2025, . 24 July 2025. 이렇게 되어있음 -> 년도와 월만 남길 것.
- media라고 되어있는 컬럼 -> affiliations로 변경
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 WallStreetJournal.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [7]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "WallStreetJournal"
# 데이터 파일 경로
FILE_PATH_1 = "/home/dslab/choi/Journal/Data/Industry/WallStreetJournal/wallstreet.csv"
FILE_PATH_2 = "/home/dslab/choi/Journal/Data/Industry/WallStreetJournal/wallstreet_content.csv"
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"WallStreetJournal.csv 생성 시작")
print(f"{'='*60}\n")

# 1. WallStreetJournal 데이터 읽기 및 병합
print("[1단계] WallStreetJournal 파일들 읽기 및 병합")

all_dfs = []

# 첫 번째 파일 읽기
if os.path.exists(FILE_PATH_1):
    try:
        df1 = pd.read_csv(FILE_PATH_1)
        all_dfs.append(df1)
        print(f"  ✓ 파일 1: {FILE_PATH_1}")
        print(f"    - 행 수: {len(df1)}")
    except Exception as e:
        print(f"  ❌ 파일 1 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 1을 찾을 수 없습니다: {FILE_PATH_1}")

# 두 번째 파일 읽기
if os.path.exists(FILE_PATH_2):
    try:
        df2 = pd.read_csv(FILE_PATH_2)
        all_dfs.append(df2)
        print(f"  ✓ 파일 2: {FILE_PATH_2}")
        print(f"    - 행 수: {len(df2)}")
    except Exception as e:
        print(f"  ❌ 파일 2 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 2를 찾을 수 없습니다: {FILE_PATH_2}")

if not all_dfs:
    print("  ❌ 읽을 수 있는 파일이 없습니다.")
    exit()

# 모든 데이터프레임 병합
df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (년월만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원:
    - 23 Aug 2025
    - 24 July 2025
    - . 28 July 2025: B.3.
    - 19 Aug 2025
    """
    if pd.isna(date_str):
        return None

    # 문자열로 변환하고 앞뒤 공백 및 특수문자 제거
    date_str = str(date_str).strip()
    # 앞의 점, 공백 제거
    date_str = date_str.lstrip('. ')

    # 월 이름 매핑
    month_names = {
        'January': '01', 'Jan': '01',
        'February': '02', 'Feb': '02',
        'March': '03', 'Mar': '03',
        'April': '04', 'Apr': '04',
        'May': '05',
        'June': '06', 'Jun': '06',
        'July': '07', 'Jul': '07',
        'August': '08', 'Aug': '08',
        'September': '09', 'Sep': '09',
        'October': '10', 'Oct': '10',
        'November': '11', 'Nov': '11',
        'December': '12', 'Dec': '12'
    }

    # "DD Month YYYY" 패턴 처리 (콜론이나 다른 텍스트 앞에 있어도 처리)
    for month_name, month_num in month_names.items():
        if month_name in date_str:
            # 연도 추출 (4자리 숫자)
            year_match = re.search(r'\b(\d{4})\b', date_str)
            if year_match:
                year = year_match.group(1)
                return f"{year}-{month_num}"

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. media를 affiliations로 변경
print(f"\n[3단계] media → affiliations 변경")

if 'media' in df.columns:
    df['affiliations'] = df['media']
    print(f"  ✓ media 컬럼을 affiliations로 변경")
else:
    # media 컬럼이 없으면 WallStreetJournal로 설정
    df['affiliations'] = JOURNAL_NAME
    print(f"  ⚠ media 컬럼이 없습니다. affiliations를 '{JOURNAL_NAME}'로 설정")

# 4. content를 abstract로 매핑
print(f"\n[4단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
elif 'abstract' not in df.columns:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None
else:
    print(f"  ✓ abstract 컬럼이 이미 존재합니다")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ WallStreetJournal.csv 생성 완료!")
print(f"{'='*60}")

WallStreetJournal.csv 생성 시작

[1단계] WallStreetJournal 파일들 읽기 및 병합
  ✓ 파일 1: /home/dslab/choi/Journal/Data/Industry/WallStreetJournal/wallstreet.csv
    - 행 수: 182
  ✓ 파일 2: /home/dslab/choi/Journal/Data/Industry/WallStreetJournal/wallstreet_content.csv
    - 행 수: 1593

✓ 병합 완료: 총 1775개 행 (2개 파일)


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: 10 Sep 2025 → 2025-09
  - 변환 완료: 1144개
  ⚠ 변환 실패: 631개

[3단계] media → affiliations 변경
  ✓ media 컬럼을 affiliations로 변경

[4단계] content → abstract 매핑
  ✓ content 컬럼을 abstract로 복사

[5단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 1775개 행
  - 제거 후: 1144개 행
  - 제거됨: 631개 행

[6단계] 연도 필터링 (2023-2025)
  - 필터링 전: 1144개 행
  - 필터링 후: 971개 행
  - 제거됨: 173개 행

[7단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 971개 행
  - 제거 후: 955개 행
  - 제거됨: 16개 행

[8단계] 최종 컬럼 선택
  ⚠ 누락된 컬럼: ['keywords']
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[9단계] 날짜별 통계
  - 총 33개의 날짜
  - 첫 날짜: 2023-01 (5개)
  - 마지막 날짜: 2025-09 (51개)

  [최근 10개 날짜]
    - 2024-12: 38개
    - 20

## 6. NewYorkTimes.csv 만들기
- 데이터 병합 -> /home/dslab/choi/WITS/원본데이터/[2025.09.25]Industry/nyt.csv , /home/dslab/choi/WITS/원본데이터/nyt_content.csv
- date 형식이 지금 23 Aug 2025, . 24 July 2025. 이렇게 되어있음 -> 년도와 월만 남길 것.
- media라고 되어있는 컬럼 -> affiliations로 변경
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 NewYorkTimes.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [6]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "NewYorkTimes"
# 데이터 파일 경로
FILE_PATH_1 = "/home/dslab/choi/WITS/원본데이터/[2025.09.25]Industry/nyt.csv"
FILE_PATH_2 = "/home/dslab/choi/WITS/원본데이터/nyt_content.csv"
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"NewYorkTimes.csv 생성 시작")
print(f"{'='*60}\n")

# 1. NewYorkTimes 데이터 읽기 및 병합
print("[1단계] NewYorkTimes 파일들 읽기 및 병합")

all_dfs = []

# 첫 번째 파일 읽기
if os.path.exists(FILE_PATH_1):
    try:
        df1 = pd.read_csv(FILE_PATH_1)
        all_dfs.append(df1)
        print(f"  ✓ 파일 1: {FILE_PATH_1}")
        print(f"    - 행 수: {len(df1)}")
    except Exception as e:
        print(f"  ❌ 파일 1 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 1을 찾을 수 없습니다: {FILE_PATH_1}")

# 두 번째 파일 읽기
if os.path.exists(FILE_PATH_2):
    try:
        df2 = pd.read_csv(FILE_PATH_2)
        all_dfs.append(df2)
        print(f"  ✓ 파일 2: {FILE_PATH_2}")
        print(f"    - 행 수: {len(df2)}")
    except Exception as e:
        print(f"  ❌ 파일 2 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 2를 찾을 수 없습니다: {FILE_PATH_2}")

if not all_dfs:
    print("  ❌ 읽을 수 있는 파일이 없습니다.")
    exit()

# 모든 데이터프레임 병합
df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (년월만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원:
    - 23 Aug 2025
    - 24 July 2025
    - . 28 July 2025: B.3.
    - 19 Aug 2025
    """
    if pd.isna(date_str):
        return None

    # 문자열로 변환하고 앞뒤 공백 및 특수문자 제거
    date_str = str(date_str).strip()
    # 앞의 점, 공백 제거
    date_str = date_str.lstrip('. ')

    # 월 이름 매핑
    month_names = {
        'January': '01', 'Jan': '01',
        'February': '02', 'Feb': '02',
        'March': '03', 'Mar': '03',
        'April': '04', 'Apr': '04',
        'May': '05',
        'June': '06', 'Jun': '06',
        'July': '07', 'Jul': '07',
        'August': '08', 'Aug': '08',
        'September': '09', 'Sep': '09',
        'October': '10', 'Oct': '10',
        'November': '11', 'Nov': '11',
        'December': '12', 'Dec': '12'
    }

    # "DD Month YYYY" 패턴 처리 (콜론이나 다른 텍스트 앞에 있어도 처리)
    for month_name, month_num in month_names.items():
        if month_name in date_str:
            # 연도 추출 (4자리 숫자)
            year_match = re.search(r'\b(\d{4})\b', date_str)
            if year_match:
                year = year_match.group(1)
                return f"{year}-{month_num}"

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. media를 affiliations로 변경
print(f"\n[3단계] media → affiliations 변경")

if 'media' in df.columns:
    df['affiliations'] = df['media']
    print(f"  ✓ media 컬럼을 affiliations로 변경")
else:
    # media 컬럼이 없으면 NewYorkTimes로 설정
    df['affiliations'] = JOURNAL_NAME
    print(f"  ⚠ media 컬럼이 없습니다. affiliations를 '{JOURNAL_NAME}'로 설정")

# 4. content를 abstract로 매핑
print(f"\n[4단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
elif 'abstract' not in df.columns:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None
else:
    print(f"  ✓ abstract 컬럼이 이미 존재합니다")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ NewYorkTimes.csv 생성 완료!")
print(f"{'='*60}")

NewYorkTimes.csv 생성 시작

[1단계] NewYorkTimes 파일들 읽기 및 병합
  ✓ 파일 1: /home/dslab/choi/WITS/원본데이터/[2025.09.25]Industry/nyt.csv
    - 행 수: 164
  ✓ 파일 2: /home/dslab/choi/WITS/원본데이터/nyt_content.csv
    - 행 수: 2780

✓ 병합 완료: 총 2944개 행 (2개 파일)


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: 23 Aug 2025 → 2025-08
  - 변환 완료: 1016개
  ⚠ 변환 실패: 1928개

[3단계] media → affiliations 변경
  ✓ media 컬럼을 affiliations로 변경

[4단계] content → abstract 매핑
  ✓ content 컬럼을 abstract로 복사

[5단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 2944개 행
  - 제거 후: 928개 행
  - 제거됨: 2016개 행

[6단계] 연도 필터링 (2023-2025)
  - 필터링 전: 928개 행
  - 필터링 후: 910개 행
  - 제거됨: 18개 행

[7단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 910개 행
  - 제거 후: 900개 행
  - 제거됨: 10개 행

[8단계] 최종 컬럼 선택
  ⚠ 누락된 컬럼: ['keywords']
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[9단계] 날짜별 통계
  - 총 33개의 날짜
  - 첫 날짜: 2023-01 (3개)
  - 마지막 날짜: 2025-09 (33개)

  [최근 10개 날짜]
    - 2024-12: 33개
    - 2025-01: 49개
    - 2025-02: 42개
    - 2025-03: 49개
    - 2025-04

## 7. CNN.csv 만들기
- 데이터 병합 -> /home/dslab/choi/Journal/Data/Industry/CNN/cnn.csv , /home/dslab/choi/Journal/Data/Industry/CNN/CNN.csv
- date 형식이 지금 3:23 PM EDT, Tue July 8, 2025 이렇게 되어있음 -> 년도와 월만 남길 것.
- affiliations로 생성 -> CNN으로
- null, 중복 제거 -> title, content 기준
- 최종 데이터 저장 CNN.csv로 필요한 컬럼만 -> date, title, abstract, keywords, affiliations
- 2023년 ~ 2025년만 남기기

In [8]:
import pandas as pd
import os
import re

# ========================================
# 설정
# ========================================
JOURNAL_NAME = "CNN"
# 데이터 파일 경로
FILE_PATH_1 = "/home/dslab/choi/Journal/Data/Industry/CNN/cnn.csv"
FILE_PATH_2 = "/home/dslab/choi/Journal/Data/Industry/CNN/CNN.csv"
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도
# ========================================

print(f"{'='*60}")
print(f"CNN.csv 생성 시작")
print(f"{'='*60}\n")

# 1. CNN 데이터 읽기 및 병합
print("[1단계] CNN 파일들 읽기 및 병합")

all_dfs = []

# 첫 번째 파일 읽기
if os.path.exists(FILE_PATH_1):
    try:
        df1 = pd.read_csv(FILE_PATH_1)
        all_dfs.append(df1)
        print(f"  ✓ 파일 1: {FILE_PATH_1}")
        print(f"    - 행 수: {len(df1)}")
    except Exception as e:
        print(f"  ❌ 파일 1 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 1을 찾을 수 없습니다: {FILE_PATH_1}")

# 두 번째 파일 읽기
if os.path.exists(FILE_PATH_2):
    try:
        df2 = pd.read_csv(FILE_PATH_2)
        all_dfs.append(df2)
        print(f"  ✓ 파일 2: {FILE_PATH_2}")
        print(f"    - 행 수: {len(df2)}")
    except Exception as e:
        print(f"  ❌ 파일 2 읽기 오류: {e}")
else:
    print(f"  ⚠ 파일 2를 찾을 수 없습니다: {FILE_PATH_2}")

if not all_dfs:
    print("  ❌ 읽을 수 있는 파일이 없습니다.")
    exit()

# 모든 데이터프레임 병합
df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. date 컬럼 처리 (년월만 남기기)
print(f"\n[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)")

if 'date' not in df.columns:
    print(f"  ❌ 'date' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

def format_date_to_month(date_str):
    """
    date를 YYYY-MM 형식으로 변환
    다양한 형식 지원:
    - 3:23 PM EDT, Tue July 8, 2025
    - 10:15 AM PST, Mon January 15, 2024
    """
    if pd.isna(date_str):
        return None

    date_str = str(date_str).strip()

    # 월 이름 매핑
    month_names = {
        'January': '01', 'Jan': '01',
        'February': '02', 'Feb': '02',
        'March': '03', 'Mar': '03',
        'April': '04', 'Apr': '04',
        'May': '05',
        'June': '06', 'Jun': '06',
        'July': '07', 'Jul': '07',
        'August': '08', 'Aug': '08',
        'September': '09', 'Sep': '09',
        'October': '10', 'Oct': '10',
        'November': '11', 'Nov': '11',
        'December': '12', 'Dec': '12'
    }

    # 월 이름과 연도 추출
    for month_name, month_num in month_names.items():
        if month_name in date_str:
            # 연도 추출 (4자리 숫자)
            year_match = re.search(r'\b(\d{4})\b', date_str)
            if year_match:
                year = year_match.group(1)
                return f"{year}-{month_num}"

    # YYYY-MM 추출 (YYYY-MM-DD, YYYY/MM/DD 등)
    match = re.search(r'(\d{4})[-/](\d{2})', date_str)
    if match:
        year = match.group(1)
        month = match.group(2)
        return f"{year}-{month}"

    # YYYY만 있는 경우
    match = re.search(r'^(\d{4})$', date_str)
    if match:
        return f"{match.group(1)}-01"

    return None

before_format = df['date'].iloc[0] if len(df) > 0 else None
df['date'] = df['date'].apply(format_date_to_month)
after_format = df['date'].iloc[0] if len(df) > 0 else None

print(f"  - 변환 예시: {before_format} → {after_format}")
print(f"  - 변환 완료: {df['date'].notna().sum()}개")
if df['date'].isna().sum() > 0:
    print(f"  ⚠ 변환 실패: {df['date'].isna().sum()}개")

# 3. content를 abstract로 매핑
print(f"\n[3단계] content → abstract 매핑")

if 'content' in df.columns:
    df['abstract'] = df['content']
    print(f"  ✓ content 컬럼을 abstract로 복사")
elif 'abstract' not in df.columns:
    print(f"  ⚠ content 컬럼이 없습니다. abstract를 빈 값으로 설정")
    df['abstract'] = None
else:
    print(f"  ✓ abstract 컬럼이 이미 존재합니다")

# 4. affiliations 칼럼 생성
print(f"\n[4단계] affiliations 생성")
df['affiliations'] = JOURNAL_NAME
print(f"  - affiliations: {JOURNAL_NAME}")

# 5. null 값 제거 (title, content 기준)
print(f"\n[5단계] Null 값 제거 (title, abstract 기준)")

# title 컬럼 존재 여부 확인
if 'title' not in df.columns:
    print(f"  ❌ 'title' 컬럼이 없습니다!")
    print(f"  현재 컬럼: {list(df.columns)}")
    exit()

before_null = len(df)
df = df.dropna(subset=['title', 'abstract'])
after_null = len(df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 6. 연도 필터링 (2023-2025)
print(f"\n[6단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(df)
df['year'] = df['date'].apply(extract_year)
df = df[df['year'].isin(VALID_YEARS)]
df = df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(df)

print(f"  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 7. 중복 제거 (title, content 기준)
print(f"\n[7단계] 중복 제거 (title, abstract 기준)")
before_dup = len(df)
df = df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 8. 최종 컬럼 선택
print(f"\n[8단계] 최종 컬럼 선택")
required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']

# 존재하는 컬럼만 선택
available_columns = [col for col in required_columns if col in df.columns]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        df[col] = None

final_df = df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")

# 9. 날짜별 통계
print(f"\n[9단계] 날짜별 통계")

if final_df['date'].notna().any():
    date_stats = final_df.groupby('date').size().sort_index()

    # 통계가 너무 많으면 요약만 표시
    if len(date_stats) > 20:
        print(f"  - 총 {len(date_stats)}개의 날짜")
        print(f"  - 첫 날짜: {date_stats.index[0]} ({date_stats.iloc[0]}개)")
        print(f"  - 마지막 날짜: {date_stats.index[-1]} ({date_stats.iloc[-1]}개)")
        print(f"\n  [최근 10개 날짜]")
        for date, count in date_stats.tail(10).items():
            print(f"    - {date}: {count}개")
    else:
        for date, count in date_stats.items():
            print(f"  - {date}: {count}개")
else:
    print(f"  - date 정보가 없습니다.")

# 10. 결과 저장
output_filename = f"{JOURNAL_NAME}.csv"
output_path = os.path.join("/home/dslab/choi/Journal/Data/Industry", output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*60}")
print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 소스: {JOURNAL_NAME}")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(3).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ CNN.csv 생성 완료!")
print(f"{'='*60}")

CNN.csv 생성 시작

[1단계] CNN 파일들 읽기 및 병합
  ✓ 파일 1: /home/dslab/choi/Journal/Data/Industry/CNN/cnn.csv
    - 행 수: 48
  ✓ 파일 2: /home/dslab/choi/Journal/Data/Industry/CNN/CNN.csv
    - 행 수: 532

✓ 병합 완료: 총 580개 행 (2개 파일)


[2단계] date 컬럼 처리 (YYYY-MM 형식으로 변환)
  - 변환 예시: nan → None
  - 변환 완료: 568개
  ⚠ 변환 실패: 12개

[3단계] content → abstract 매핑
  ✓ content 컬럼을 abstract로 복사

[4단계] affiliations 생성
  - affiliations: CNN

[5단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 580개 행
  - 제거 후: 532개 행
  - 제거됨: 48개 행

[6단계] 연도 필터링 (2023-2025)
  - 필터링 전: 532개 행
  - 필터링 후: 496개 행
  - 제거됨: 36개 행

[7단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 496개 행
  - 제거 후: 458개 행
  - 제거됨: 38개 행

[8단계] 최종 컬럼 선택
  ⚠ 누락된 컬럼: ['keywords']
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[9단계] 날짜별 통계
  - 2023-01: 216개
  - 2024-01: 177개
  - 2025-01: 65개

✓ 최종 저장 완료!
✓ 파일 경로: /home/dslab/choi/Journal/Data/Industry/CNN.csv
✓ 최종 행 수: 458개
✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
✓ 소스: CNN
✓ 연도 범위: 2

# Academia 합치기
- DSS, EJIS, IAM, ICIS, ISJ, ISR, JAIS, JIT, JMIS, JSIS, MISQ 총 12개 저널과 컨퍼런스로 이루어짐
- date, title, abstract, keywords, affiliations 칼럼으로 이루어짐
- date는 값이 다 다를 수 있음
- date 2023년 ~ 2025년도 인지 다시 한번 확인
- Title, abstract을 기준으로 Null, 중복제거 재진행
- 마지막에 value_counts()로 date, affiliations로 찍어보기

In [4]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
OUTPUT_NAME = "Academia"
DATA_DIR = "/home/dslab/choi/Journal/Data/Academia/"  # 각 저널 CSV가 있는 디렉토리
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도

# 병합할 저널/컨퍼런스 목록
JOURNALS = [
    "DSS", "EJIS", "IAM", "ICIS", "ISJ", "ISR",
    "JAIS", "JIT", "JMIS", "JSIS", "MISQ", "HICSS"
]
# ========================================

print(f"{'='*60}")
print(f"Academia 데이터 병합 시작")
print(f"{'='*60}\n")

# 1. 모든 저널 데이터 읽기 및 병합
print("[1단계] 저널/컨퍼런스 데이터 읽기 및 병합")

all_dfs = []
found_journals = []
missing_journals = []

for journal in JOURNALS:
    file_path = os.path.join(DATA_DIR, f"{journal}.csv")

    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            all_dfs.append(df)
            found_journals.append(journal)
            print(f"  ✓ {journal:10s} -> {len(df):6d}개 행")
        except Exception as e:
            print(f"  ❌ {journal:10s} -> 읽기 오류: {e}")
            missing_journals.append(journal)
    else:
        print(f"  ⚠ {journal:10s} -> 파일 없음")
        missing_journals.append(journal)

if not all_dfs:
    print("\n❌ 읽을 수 있는 파일이 없습니다.")
    exit()

print(f"\n✓ 발견된 저널: {len(found_journals)}개 - {', '.join(found_journals)}")
if missing_journals:
    print(f"⚠ 누락된 저널: {len(missing_journals)}개 - {', '.join(missing_journals)}")

# 모든 데이터프레임 병합
combined_df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. 필수 컬럼 확인
print(f"\n[2단계] 필수 컬럼 확인")

required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']
available_columns = [col for col in required_columns if col in combined_df.columns]
missing_columns = [col for col in required_columns if col not in combined_df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        combined_df[col] = None
    print(f"  → 누락된 컬럼을 None으로 생성했습니다")

print(f"  ✓ 컬럼: {list(combined_df.columns)}")

# 3. 연도 필터링 (2023-2025)
print(f"\n[3단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(combined_df)
combined_df['year'] = combined_df['date'].apply(extract_year)

# 연도별 분포 확인
year_distribution = combined_df['year'].value_counts().sort_index()
print(f"  [연도별 분포 (필터링 전)]")
for year, count in year_distribution.items():
    status = "✓" if year in VALID_YEARS else "✗"
    print(f"    {status} {year}: {count}개")

# 유효한 연도만 필터링
combined_df = combined_df[combined_df['year'].isin(VALID_YEARS)]
combined_df = combined_df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(combined_df)

print(f"\n  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 4. Null 값 제거 (title, abstract 기준)
print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

before_null = len(combined_df)
combined_df = combined_df.dropna(subset=['title', 'abstract'])
after_null = len(combined_df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 5. 중복 제거 (title, abstract 기준)
print(f"\n[5단계] 중복 제거 (title, abstract 기준)")

before_dup = len(combined_df)
combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(combined_df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 6. 최종 컬럼 선택
print(f"\n[6단계] 최종 컬럼 선택")

final_df = combined_df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")
print(f"  - 최종 행 수: {len(final_df)}개")

# 7. 통계 확인
print(f"\n{'='*60}")
print(f"[7단계] 데이터 통계")
print(f"{'='*60}\n")

# affiliations별 통계
print("■ Affiliations별 통계:")
affiliations_stats = final_df['affiliations'].value_counts().sort_index()
total_affiliations = len(final_df)
for affiliation, count in affiliations_stats.items():
    percentage = (count / total_affiliations) * 100
    print(f"  - {affiliation:10s}: {count:6d}개 ({percentage:5.2f}%)")
print(f"  {'─'*40}")
print(f"  - {'합계':10s}: {total_affiliations:6d}개 (100.00%)")

# date별 통계 (연-월 기준)
print(f"\n■ Date별 통계:")

def format_date_for_stats(date_str):
    """date를 YYYY-MM 형식으로 통일"""
    if pd.isna(date_str):
        return 'Unknown'
    date_str = str(date_str)
    # YYYY-MM 추출
    match = re.search(r'(\d{4})-?(\d{2})?', date_str)
    if match:
        year = match.group(1)
        month = match.group(2) if match.group(2) else '01'
        return f"{year}-{month}"
    return date_str

final_df['date_formatted'] = final_df['date'].apply(format_date_for_stats)
date_stats = final_df.groupby('date_formatted').size().sort_index()

# 날짜가 너무 많으면 연도별로 집계
if len(date_stats) > 36:  # 3년 * 12개월
    print(f"  총 {len(date_stats)}개의 날짜 (연도별 요약)")

    # 연도별 집계
    year_stats = {}
    for date, count in date_stats.items():
        year = date[:4] if len(date) >= 4 else 'Unknown'
        year_stats[year] = year_stats.get(year, 0) + count

    for year in sorted(year_stats.keys()):
        count = year_stats[year]
        percentage = (count / total_affiliations) * 100
        print(f"    - {year}: {count:6d}개 ({percentage:5.2f}%)")
else:
    # 전체 날짜 표시
    for date, count in date_stats.items():
        percentage = (count / total_affiliations) * 100
        print(f"  - {date}: {count:6d}개 ({percentage:5.2f}%)")

# date_formatted 임시 컬럼 제거
final_df = final_df.drop('date_formatted', axis=1)

# 8. 결과 저장
print(f"\n{'='*60}")
print(f"[8단계] 결과 저장")
print(f"{'='*60}\n")

output_filename = f"{OUTPUT_NAME}.csv"
output_path = os.path.join(DATA_DIR, output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 저널 수: {len(found_journals)}개")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(5).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ Academia.csv 생성 완료!")
print(f"{'='*60}")

Academia 데이터 병합 시작

[1단계] 저널/컨퍼런스 데이터 읽기 및 병합
  ✓ DSS        ->    212개 행
  ✓ EJIS       ->    165개 행
  ✓ IAM        ->    270개 행
  ✓ ICIS       ->    835개 행
  ✓ ISJ        ->    148개 행
  ✓ ISR        ->     58개 행
  ✓ JAIS       ->    166개 행
  ✓ JIT        ->     50개 행
  ✓ JMIS       ->    128개 행
  ✓ JSIS       ->     65개 행
  ✓ MISQ       ->    194개 행
  ✓ HICSS      ->   2098개 행

✓ 발견된 저널: 12개 - DSS, EJIS, IAM, ICIS, ISJ, ISR, JAIS, JIT, JMIS, JSIS, MISQ, HICSS

✓ 병합 완료: 총 4389개 행 (12개 파일)


[2단계] 필수 컬럼 확인
  ✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[3단계] 연도 필터링 (2023-2025)
  [연도별 분포 (필터링 전)]
    ✓ 2023.0: 1517개
    ✓ 2024.0: 1698개
    ✓ 2025.0: 1149개

  - 필터링 전: 4389개 행
  - 필터링 후: 4364개 행
  - 제거됨: 25개 행

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 4364개 행
  - 제거 후: 4364개 행
  - 제거됨: 0개 행

[5단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 4364개 행
  - 제거 후: 4363개 행
  - 제거됨: 1개 행

[6단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
  - 최종

In [5]:
df=pd.read_csv('/home/dslab/choi/Journal/Data/Academia/Academia.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4363 entries, 0 to 4362
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          4363 non-null   object
 1   title         4363 non-null   object
 2   abstract      4363 non-null   object
 3   keywords      4153 non-null   object
 4   affiliations  4363 non-null   object
dtypes: object(5)
memory usage: 170.6+ KB


# Industry 합치기
- BBC, CNN, NewYorkTimes, TechCrunch, TheGuardian, TheVerge, WallStreetJournal 총 7개 media로 이루어짐
- date, title, abstract, keywords, affiliations 칼럼으로 이루어짐
- date는 값이 다 다를 수 있음
- date 2023년 ~ 2025년도 인지 다시 한번 확인
- title, abstract을 기준으로 Null, 중복제거 재진행
- 마지막에 value_counts()로 date, affiliations로 찍어보기

In [7]:
import pandas as pd
import os
import re
from glob import glob

# ========================================
# 설정
# ========================================
OUTPUT_NAME = "Industry"
DATA_DIR = "/home/dslab/choi/Journal/Data/Industry/"  # 각 미디어 CSV가 있는 디렉토리
VALID_YEARS = [2023, 2024, 2025]  # 유효한 연도

# 병합할 미디어 목록
MEDIA_LIST = [
    "NewYorkTimes", "TechCrunch",
    "TheGuardian", "TheVerge", "WallStreetJournal"
]
# ========================================

print(f"{'='*60}")
print(f"Industry 데이터 병합 시작")
print(f"{'='*60}\n")

# 1. 모든 미디어 데이터 읽기 및 병합
print("[1단계] 미디어 데이터 읽기 및 병합")

all_dfs = []
found_media = []
missing_media = []

for media in MEDIA_LIST:
    file_path = os.path.join(DATA_DIR, f"{media}.csv")

    if os.path.exists(file_path):
        try:
            df = pd.read_csv(file_path)
            all_dfs.append(df)
            found_media.append(media)
            print(f"  ✓ {media:20s} -> {len(df):6d}개 행")
        except Exception as e:
            print(f"  ❌ {media:20s} -> 읽기 오류: {e}")
            missing_media.append(media)
    else:
        print(f"  ⚠ {media:20s} -> 파일 없음")
        missing_media.append(media)

if not all_dfs:
    print("\n❌ 읽을 수 있는 파일이 없습니다.")
    exit()

print(f"\n✓ 발견된 미디어: {len(found_media)}개 - {', '.join(found_media)}")
if missing_media:
    print(f"⚠ 누락된 미디어: {len(missing_media)}개 - {', '.join(missing_media)}")

# 모든 데이터프레임 병합
combined_df = pd.concat(all_dfs, ignore_index=True)
print(f"\n✓ 병합 완료: 총 {len(combined_df)}개 행 ({len(all_dfs)}개 파일)")

print(f"\n{'='*60}")

# 2. 필수 컬럼 확인
print(f"\n[2단계] 필수 컬럼 확인")

required_columns = ['date', 'title', 'abstract', 'keywords', 'affiliations']
available_columns = [col for col in required_columns if col in combined_df.columns]
missing_columns = [col for col in required_columns if col not in combined_df.columns]

if missing_columns:
    print(f"  ⚠ 누락된 컬럼: {missing_columns}")
    for col in missing_columns:
        combined_df[col] = None
    print(f"  → 누락된 컬럼을 None으로 생성했습니다")

print(f"  ✓ 컬럼: {list(combined_df.columns)}")

# 3. 연도 필터링 (2023-2025)
print(f"\n[3단계] 연도 필터링 (2023-2025)")

def extract_year(date_str):
    """date에서 연도 추출"""
    if pd.isna(date_str):
        return None
    date_str = str(date_str)
    match = re.search(r'(\d{4})', date_str)
    if match:
        return int(match.group(1))
    return None

before_year_filter = len(combined_df)
combined_df['year'] = combined_df['date'].apply(extract_year)

# 연도별 분포 확인
year_distribution = combined_df['year'].value_counts().sort_index()
print(f"  [연도별 분포 (필터링 전)]")
for year, count in year_distribution.items():
    status = "✓" if year in VALID_YEARS else "✗"
    print(f"    {status} {year}: {count}개")

# 유효한 연도만 필터링
combined_df = combined_df[combined_df['year'].isin(VALID_YEARS)]
combined_df = combined_df.drop('year', axis=1)  # 임시 컬럼 제거
after_year_filter = len(combined_df)

print(f"\n  - 필터링 전: {before_year_filter}개 행")
print(f"  - 필터링 후: {after_year_filter}개 행")
print(f"  - 제거됨: {before_year_filter - after_year_filter}개 행")

# 4. Null 값 제거 (title, abstract 기준)
print(f"\n[4단계] Null 값 제거 (title, abstract 기준)")

before_null = len(combined_df)
combined_df = combined_df.dropna(subset=['title', 'abstract'])
after_null = len(combined_df)
print(f"  - 제거 전: {before_null}개 행")
print(f"  - 제거 후: {after_null}개 행")
print(f"  - 제거됨: {before_null - after_null}개 행")

# 5. 중복 제거 (title, abstract 기준)
print(f"\n[5단계] 중복 제거 (title, abstract 기준)")

before_dup = len(combined_df)
combined_df = combined_df.drop_duplicates(subset=['title', 'abstract'], keep='first')
after_dup = len(combined_df)
print(f"  - 제거 전: {before_dup}개 행")
print(f"  - 제거 후: {after_dup}개 행")
print(f"  - 제거됨: {before_dup - after_dup}개 행")

# 6. 최종 컬럼 선택
print(f"\n[6단계] 최종 컬럼 선택")

final_df = combined_df[required_columns].copy()
print(f"  - 최종 컬럼: {list(final_df.columns)}")
print(f"  - 최종 행 수: {len(final_df)}개")

# 7. 통계 확인
print(f"\n{'='*60}")
print(f"[7단계] 데이터 통계")
print(f"{'='*60}\n")

# affiliations별 통계
print("■ Affiliations별 통계:")
affiliations_stats = final_df['affiliations'].value_counts().sort_index()
total_affiliations = len(final_df)
for affiliation, count in affiliations_stats.items():
    percentage = (count / total_affiliations) * 100
    print(f"  - {affiliation:20s}: {count:6d}개 ({percentage:5.2f}%)")
print(f"  {'─'*50}")
print(f"  - {'합계':20s}: {total_affiliations:6d}개 (100.00%)")

# date별 통계 (연-월 기준)
print(f"\n■ Date별 통계:")

def format_date_for_stats(date_str):
    """date를 YYYY-MM 형식으로 통일"""
    if pd.isna(date_str):
        return 'Unknown'
    date_str = str(date_str)
    # YYYY-MM 추출
    match = re.search(r'(\d{4})-?(\d{2})?', date_str)
    if match:
        year = match.group(1)
        month = match.group(2) if match.group(2) else '01'
        return f"{year}-{month}"
    return date_str

final_df['date_formatted'] = final_df['date'].apply(format_date_for_stats)
date_stats = final_df.groupby('date_formatted').size().sort_index()

# 날짜가 너무 많으면 연도별로 집계
if len(date_stats) > 36:  # 3년 * 12개월
    print(f"  총 {len(date_stats)}개의 날짜 (연도별 요약)")

    # 연도별 집계
    year_stats = {}
    for date, count in date_stats.items():
        year = date[:4] if len(date) >= 4 else 'Unknown'
        year_stats[year] = year_stats.get(year, 0) + count

    for year in sorted(year_stats.keys()):
        count = year_stats[year]
        percentage = (count / total_affiliations) * 100
        print(f"    - {year}: {count:6d}개 ({percentage:5.2f}%)")
else:
    # 전체 날짜 표시
    for date, count in date_stats.items():
        percentage = (count / total_affiliations) * 100
        print(f"  - {date}: {count:6d}개 ({percentage:5.2f}%)")

# date_formatted 임시 컬럼 제거
final_df = final_df.drop('date_formatted', axis=1)

# 8. 결과 저장
print(f"\n{'='*60}")
print(f"[8단계] 결과 저장")
print(f"{'='*60}\n")

output_filename = f"{OUTPUT_NAME}.csv"
output_path = os.path.join(DATA_DIR, output_filename)
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✓ 최종 저장 완료!")
print(f"✓ 파일 경로: {output_path}")
print(f"✓ 최종 행 수: {len(final_df)}개")
print(f"✓ 컬럼: {list(final_df.columns)}")
print(f"✓ 미디어 수: {len(found_media)}개")
print(f"✓ 연도 범위: {min(VALID_YEARS)} - {max(VALID_YEARS)}")

# 샘플 데이터 미리보기
print(f"\n[데이터 미리보기]")
if len(final_df) > 0:
    print(final_df.head(5).to_string())
else:
    print("  - 데이터가 없습니다.")

print(f"\n{'='*60}")
print(f"✓ Industry.csv 생성 완료!")
print(f"{'='*60}")

Industry 데이터 병합 시작

[1단계] 미디어 데이터 읽기 및 병합
  ✓ NewYorkTimes         ->    900개 행
  ✓ TechCrunch           ->    715개 행
  ✓ TheGuardian          ->   1986개 행
  ✓ TheVerge             ->   4584개 행
  ✓ WallStreetJournal    ->    955개 행

✓ 발견된 미디어: 5개 - NewYorkTimes, TechCrunch, TheGuardian, TheVerge, WallStreetJournal

✓ 병합 완료: 총 9140개 행 (5개 파일)


[2단계] 필수 컬럼 확인
  ✓ 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']

[3단계] 연도 필터링 (2023-2025)
  [연도별 분포 (필터링 전)]
    ✓ 2023: 2318개
    ✓ 2024: 3006개
    ✓ 2025: 3816개

  - 필터링 전: 9140개 행
  - 필터링 후: 9140개 행
  - 제거됨: 0개 행

[4단계] Null 값 제거 (title, abstract 기준)
  - 제거 전: 9140개 행
  - 제거 후: 9140개 행
  - 제거됨: 0개 행

[5단계] 중복 제거 (title, abstract 기준)
  - 제거 전: 9140개 행
  - 제거 후: 9140개 행
  - 제거됨: 0개 행

[6단계] 최종 컬럼 선택
  - 최종 컬럼: ['date', 'title', 'abstract', 'keywords', 'affiliations']
  - 최종 행 수: 9140개

[7단계] 데이터 통계

■ Affiliations별 통계:
  - New York Times      :    900개 ( 9.85%)
  - TechCrunch          :    715개 ( 7.82%)
  - TheGuardian         :